# Consolidated Data Pipeline for Member Data - for Training data

Prepare by Tan Bee Hoon

16 Aug 2020

## Import Package

In [ ]:
import random 
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import numpy as np 
from numpy import unique
from numpy import where

import pickle
from pandas import to_pickle
from pandas import read_pickle

from datetime import date

import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_colwidth', None)

## Set Global Variable

### Set input filenames

In [ ]:
FILE_DIAG = "tbl_Booking_Diagnosis.csv"
FILE_PRESCRIPTION = "tbl_Booking_Prescription.csv"
FILE_SYMPTOMS = "tbl_Booking_Symptoms.csv"
FILE_BOOKING = "tbl_Booking.csv"
FILE_PATIENT = "tbl_Patient.csv"

###  Set for Testing or Training

In [ ]:
# If its for test data (TESTING=1), to apply for model training, set min value as 0
# This is because for Testing hold out dataset purpose, will have to include previously bookings for existing members
TESTING = 0

### Set for Member or Booking Modeling

In [ ]:
# For member model = 1, else for booking model = 0 
MODEL_TYPE_MEMBER = 1

### Set previous booking related source file (for Testing)

In [ ]:
if TESTING == 1 and MODEL_TYPE_MEMBER == 1:
    FILE_DIAG_OLD = "tbl_Booking_Diagnosis.csv"
    FILE_PRESCRIPTION_OLD = "tbl_Booking_Prescription.csv"
    FILE_SYMPTOMS_OLD= "tbl_Booking_Symptoms.csv"
    FILE_BOOKING_OLD = "tbl_Booking.csv"

### Set minium count for feature considerations

In [ ]:
if TESTING == 0: 
    MIN_VAL_SYM = 5
    MIN_VAL_DIAG = 10
elif TESTING == 1:
    MIN_VAL_SYM = 0
    MIN_VAL_DIAG = 0
else:
    print("Decide whether this data prep is for training or testing.")

## Helper Function

In [ ]:
def count_unique(df):
    # List of categorical columns
    #cat_cols = df.select_dtypes('object').columns
    uniq_cols = df.columns
    
    list_count = []
    list_name = []
    
    for col in uniq_cols:
        # Formatting
        list_name.append(col)
        uniqueValues = df[col].nunique()
        list_count.append(uniqueValues)
        #print(str(col) + " has " + str(uniqueValues) + " values ")
        
    df_count = pd.DataFrame({"name" : list_name, "uniquecount" : list_count})
        
    return df_count

## Step1 - Intro Prep & Clean

* Extract list of patient
* Extract list of booking 
* Concatenate, Prep & Clean Booking Diagnosis
* Concatenate, Prep & Clean Booking Prescription
* Concatenate, Prep & Clean Booking Symptoms

### Extract list of patient

In [ ]:
# Import Source Paitent Data
df_patient = pd.read_csv(FILE_PATIENT)
patient_list = df_patient["mem_id"].tolist()
print("List of patient: ", len(patient_list))

List of patient:  5000


### Extract list of booking

In [ ]:
# For new booking list
df_booking = pd.read_csv(FILE_BOOKING)
print("BEFORE: ", "\n")
print(df_booking.info(), "\n")
df_booking = df_booking[df_booking['mem_id'].isin(patient_list)]
print("AFTER: ", "\n")
print(df_booking.info(), "\n")
booking_list = df_booking['booking_id'].tolist()
print("Number of booking: ", len(booking_list))

BEFORE:  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10221 entries, 0 to 10220
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
dtypes: bool(2), int64(1), object(5)
memory usage: 499.2+ KB
None 

AFTER:  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 102

In [ ]:
# For old booking list (only required for member holdout dataset testing)
if TESTING == 1 and MODEL_TYPE_MEMBER == 1:
    df_booking_old = pd.read_csv(FILE_BOOKING_OLD)
    print(FILE_BOOKING_OLD)
    print("BEFORE: ","\n")
    print(df_booking_old.info(), "\n")
    df_booking_old = df_booking_old[df_booking_old['mem_id'].isin(patient_list)]
    print("AFTER: ", "\n")
    print(df_booking_old.info(), "\n")
    booking_old_list = df_booking_old["booking_id"].tolist()
    print("Number of booking from previous: ", len(booking_old_list))
    print("Number of member with previous booking: ",  len(df_booking_old['mem_id'].unique()))

### Check overlapping booking ID between previous and new

In [ ]:
if TESTING == 1 and MODEL_TYPE_MEMBER ==1 :
    set1 = set(booking_list)
    set2 = set(booking_old_list)
    newList = list(set1.intersection(set2))
    print("Intersection of the lists is:", newList)

if there is no intersection, it means the new dataset did not contain previous booking.

### Concatenate new and old booking list - for Testing

In [ ]:
## Concatenate Booking ID
if TESTING == 1 and MODEL_TYPE_MEMBER == 1:
    booking_list = booking_list + booking_old_list

### Save Booking List 

In [ ]:
# Export data 
open_file = open("step1_booking_list.pkl", "wb")
pickle.dump(booking_list, open_file)
open_file.close()

## Step1a - Prep & Clean Booking Diagnosis

### Import booking diagnosis data

In [ ]:
# import new booking diagnosis
print("Input: ", FILE_DIAG, "\n") 
df_booking_diagnosis = pd.read_csv(FILE_DIAG)
df_booking_diagnosis = df_booking_diagnosis[df_booking_diagnosis['booking_id'].isin(booking_list)]
print(df_booking_diagnosis.info(), "\n") 
print("Shape of data: ", df_booking_diagnosis.shape)

Input:  tbl_Booking_Diagnosis.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12749 entries, 1 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        12749 non-null  object
 1   MDDiagnosis_code  12749 non-null  object
 2   MDDiagnosis_name  12749 non-null  object
 3   chronic_group     12749 non-null  object
dtypes: object(4)
memory usage: 498.0+ KB
None 

Shape of data:  (12749, 4)


In [ ]:
# import old booking diagnosis
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_DIAG_OLD, "\n") 
    df_booking_diagnosis_old = pd.read_csv(FILE_DIAG_OLD)
    df_booking_diagnosis_old = df_booking_diagnosis_old[df_booking_diagnosis_old['booking_id'].isin(booking_list)]
    print(df_booking_diagnosis_old.info(), "\n")
    print("Shape of data: ", df_booking_diagnosis_old.shape)


In [ ]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1 :
    df_booking_diagnosis = pd.concat([df_booking_diagnosis, df_booking_diagnosis_old], axis=0)
    print("COMBINED: ")
    print(df_booking_diagnosis.info(), "\n")
    print("Shape of data: ", df_booking_diagnosis.shape)

### Summary Statistics

In [ ]:
print("Basic Data Summary")
df_booking_diagnosis.describe().transpose()

Basic Data Summary


,count,unique,top,freq
booking_id,12749,10221,89206016-CAA8-4E18-BD64-0EFAF312DAB9,6
MDDiagnosis_code,12749,646,R51,1217
MDDiagnosis_name,12749,651,Headache,1217
chronic_group,12749,11,-,10996


In [ ]:
print("Unique value count for each column")
count_unique(df_booking_diagnosis)

Unique value count for each column


,name,uniquecount
0,booking_id,10221
1,MDDiagnosis_code,646
2,MDDiagnosis_name,651
3,chronic_group,11


### Missing Value Checks

In [ ]:
print("number of null: ")
df_booking_diagnosis.isnull().sum()

number of null: 


booking_id          0
MDDiagnosis_code    0
MDDiagnosis_name    0
chronic_group       0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Basic Cleaning

#### Change text to all lower case

In [ ]:
df_booking_diagnosis['MDDiagnosis_name'] = df_booking_diagnosis['MDDiagnosis_name'].str.lower()

#### Remove comma, replace with semi-colon 

In [ ]:
df_booking_diagnosis.MDDiagnosis_name = df_booking_diagnosis.MDDiagnosis_name.str.replace(',', ';')

#### Check and remove duplicates

In [ ]:
print("number of duplicate to drop: ", df_booking_diagnosis.duplicated().sum())

number of duplicate to drop:  7


In [ ]:
# to drop duplicates 
if df_booking_diagnosis.duplicated().sum() > 0:
    df_booking_diagnosis = df_booking_diagnosis.drop_duplicates()

### Update Data Types

#### Change data types for categorical data

In [ ]:
df_booking_diagnosis['MDDiagnosis_code'] = df_booking_diagnosis['MDDiagnosis_code'].astype("category")

In [ ]:
df_booking_diagnosis['MDDiagnosis_name'] = df_booking_diagnosis['MDDiagnosis_name'].astype("category")

In [ ]:
df_booking_diagnosis['chronic_group'] = df_booking_diagnosis['chronic_group'].astype("category")

In [ ]:
df_booking_diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12742 entries, 1 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   booking_id        12742 non-null  object  
 1   MDDiagnosis_code  12742 non-null  category
 2   MDDiagnosis_name  12742 non-null  category
 3   chronic_group     12742 non-null  category
dtypes: category(3), object(1)
memory usage: 304.1+ KB


### Check OntoOne Mapping

In [ ]:
g = df_booking_diagnosis[['MDDiagnosis_code', 'MDDiagnosis_name']].groupby('MDDiagnosis_name')
counts = g.transform(lambda x: len(x.unique()))
df_booking_diagnosis["count_map"] = counts

In [ ]:
print("These are the records with more than one mapping for name vs code, please verify:")

These are the records with more than one mapping for name vs code, please verify:


In [ ]:
pd.set_option('display.max_colwidth', None)
df_booking_diagnosis.loc[df_booking_diagnosis["count_map"]>1,:]

,booking_id,MDDiagnosis_code,MDDiagnosis_name,chronic_group,count_map


<span class="mark">IMPORTANT: 
Please verify data if above shows that there is duplicate mapping.</span> 

#### Remove Redundant Mapping

In [ ]:
df_booking_diagnosis = df_booking_diagnosis.drop_duplicates(subset=["booking_id", "MDDiagnosis_name"])

In [ ]:
df_booking_diagnosis.drop(["count_map"], inplace = True, axis = 1)

In [ ]:
df_booking_diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12742 entries, 1 to 65738
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   booking_id        12742 non-null  object  
 1   MDDiagnosis_code  12742 non-null  category
 2   MDDiagnosis_name  12742 non-null  category
 3   chronic_group     12742 non-null  category
dtypes: category(3), object(1)
memory usage: 304.1+ KB


In [ ]:
print("Unique value count for each column")
count_unique(df_booking_diagnosis)

Unique value count for each column


,name,uniquecount
0,booking_id,10221
1,MDDiagnosis_code,646
2,MDDiagnosis_name,646
3,chronic_group,11


### Data Aggregation

#### Check Unique Diagnosis

In [ ]:
print(df_booking_diagnosis["MDDiagnosis_name"].nunique())

646


In [ ]:
print(df_booking_diagnosis['MDDiagnosis_name'].value_counts().head(20))

headache                                                                1217
gastroenteritis and colitis of unspecified origin                       1144
acute upper respiratory infection; unspecified                           812
other complications following immunization; not elsewhere classified     502
dysmenorrhoea; unspecified                                               502
myalgia                                                                  351
muscle strain                                                            319
rash and other nonspecific skin eruption                                 295
gastritis; unspecified                                                   277
low back pain                                                            252
vasomotor and allergic rhinitis                                          248
tension-type headache                                                    240
essential (primary) hypertension                                         239

In [ ]:
print(df_booking_diagnosis['MDDiagnosis_name'].value_counts(normalize=True).head(20))

headache                                                                0.095511
gastroenteritis and colitis of unspecified origin                       0.089782
acute upper respiratory infection; unspecified                          0.063726
other complications following immunization; not elsewhere classified    0.039397
dysmenorrhoea; unspecified                                              0.039397
myalgia                                                                 0.027547
muscle strain                                                           0.025035
rash and other nonspecific skin eruption                                0.023152
gastritis; unspecified                                                  0.021739
low back pain                                                           0.019777
vasomotor and allergic rhinitis                                         0.019463
tension-type headache                                                   0.018835
essential (primary) hyperten

#### Check Unique Chronic

In [ ]:
print(df_booking_diagnosis["chronic_group"].nunique())

11


In [ ]:
print(df_booking_diagnosis["chronic_group"].value_counts())

-                                    10989
Vasomotor and allergic rhinitis        377
Hypertension                           241
Gastro-oesophageal reflux disease      200
Atopic dermatitis                      195
Migraine                               190
Hyperlipidaemia                        176
Asthma                                 148
Gout                                   125
Diabetes                                57
Thyroid                                 44
Name: chronic_group, dtype: int64


In [ ]:
print(df_booking_diagnosis["chronic_group"].value_counts(normalize=True)*100)

-                                    86.242348
Vasomotor and allergic rhinitis       2.958719
Hypertension                          1.891383
Gastro-oesophageal reflux disease     1.569612
Atopic dermatitis                     1.530372
Migraine                              1.491132
Hyperlipidaemia                       1.381259
Asthma                                1.161513
Gout                                  0.981008
Diabetes                              0.447340
Thyroid                               0.345315
Name: chronic_group, dtype: float64


#### Rollup

The purpose here is to roll up all the diagnose of the same booking into one roll, while retaining all diagnosis info, by converting the information into a list.

In [ ]:
# get a dict of unique ID from booking  
booking_ID_list = set(df_booking_diagnosis["booking_id"])

In [ ]:
diag_dup_list = pd.DataFrame(columns=["booking_id","diagnosis", "count_diagnosis"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_diagnosis[df_booking_diagnosis["booking_id"] == x]
    diag_list_tmp = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDDiagnosis_name"] is not np.nan:
            if (row["MDDiagnosis_name"] not in diag_list_tmp):
                diag_list_tmp.append(row["MDDiagnosis_name"])
             
    diag_dup_list.at[x, "booking_id"] = x
    diag_dup_list.at[x, "diagnosis"] = diag_list_tmp
    diag_dup_list.at[x, "count_diagnosis"] = int(len(diag_list_tmp))

In [ ]:
diagcode_dup_list = pd.DataFrame(columns=["booking_id","diagnosis_code", "count_diagnosis_code"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_diagnosis[df_booking_diagnosis["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDDiagnosis_code"] is not np.nan:
            if (row["MDDiagnosis_code"] not in tmp_list):
                tmp_list.append(row["MDDiagnosis_code"])
             
    diagcode_dup_list.at[x, "booking_id"] = x
    diagcode_dup_list.at[x, "diagnosis_code"] = tmp_list
    diagcode_dup_list.at[x, "count_diagnosis_code"] = int(len(tmp_list))

In [ ]:
chronic_dup_list = pd.DataFrame(columns=["booking_id","chronic", "count_chronic"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_diagnosis[df_booking_diagnosis["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["chronic_group"] is not np.nan and row["chronic_group"] is not "-":
            if (row["chronic_group"] not in tmp_list):
                tmp_list.append(row["chronic_group"])
        
    chronic_dup_list.at[x, "booking_id"] = x
    chronic_dup_list.at[x, "chronic"] = tmp_list
    chronic_dup_list.at[x, "count_chronic"] = int(len(tmp_list))

In [ ]:
# Perform an inner join between the temp diagnosis name and code tables
combine_data_diag = pd.merge(diagcode_dup_list, diag_dup_list, how="inner", on=["booking_id"])

In [ ]:
# Perform an inner join between the chronic temp table & above
combine_data_diag_chronic = pd.merge(combine_data_diag, chronic_dup_list, how="inner", on=["booking_id"])

### Finalise Prep 

In [ ]:
final_df_booking_diagnosis = combine_data_diag_chronic[["booking_id","diagnosis_code","diagnosis","chronic", 
                                      "count_diagnosis", "count_chronic"]]

In [ ]:
final_df_booking_diagnosis.columns = ['booking_id', 'diagnosis_code_list', 'diagnosis_list', 'chronic_list',
       'count_diagnosis', 'count_chronic']

In [ ]:
final_df_booking_diagnosis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           10221 non-null  object
 1   diagnosis_code_list  10221 non-null  object
 2   diagnosis_list       10221 non-null  object
 3   chronic_list         10221 non-null  object
 4   count_diagnosis      10221 non-null  object
 5   count_chronic        10221 non-null  object
dtypes: object(6)
memory usage: 559.0+ KB


In [ ]:
final_df_booking_diagnosis.head(5)

,booking_id,diagnosis_code_list,diagnosis_list,chronic_list,count_diagnosis,count_chronic
0,CAA7BA7B-B12F-4E1E-9A7A-2284A72BF9CB,[A09.9],[gastroenteritis and colitis of unspecified origin],[],1,0
1,2532BFF4-5C2F-4256-A738-EB38EB1A6139,[K07.6],[temporomandibular joint disorders],[],1,0
2,706B0106-A554-4AF8-9E0B-83E339DB6CA3,[N94.6],[dysmenorrhoea; unspecified],[],1,0
3,E6952025-2750-4123-8963-4D5E9D1F0DB2,[R51],[headache],[],1,0
4,8F396092-C568-4F35-9A60-22C2471122F7,"[N94.6, R51]","[dysmenorrhoea; unspecified, headache]",[],2,0


### Save intermediate Output 

In [ ]:
# Export data 
final_df_booking_diagnosis.to_pickle("step1_booking_diagnosis.pkl")

<span class="burk">**COMPLETE STEP1 FOR BOOKING DIAGNOSIS**</span>

## Step1b - Prep & Clean Booking Prescription

### Import booking Prescription data

In [ ]:
# import new booking prescription
print("Input: ", FILE_PRESCRIPTION, "\n") 
df_booking_presciption = pd.read_csv(FILE_PRESCRIPTION)
df_booking_presciption = df_booking_presciption[df_booking_presciption['booking_id'].isin(booking_list)]
print(df_booking_presciption.info(), "\n") 
print("Shape of data: ", df_booking_presciption.shape)

Input:  tbl_Booking_Prescription.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20241 entries, 0 to 20240
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   booking_id               20241 non-null  object
 1   MDMedication_ID          20241 non-null  object
 2   MDMedication_name        20241 non-null  object
 3   is_fulfilled_externally  20241 non-null  bool  
 4   prescription_quantity    20241 non-null  int64 
 5   prescription_status      20241 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 968.6+ KB
None 

Shape of data:  (20241, 6)


In [ ]:
# import old booking prescription
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_PRESCRIPTION_OLD, "\n") 
    df_booking_presciption_old = pd.read_csv(FILE_PRESCRIPTION_OLD)
    df_booking_presciption_old = df_booking_presciption_old[df_booking_presciption_old['booking_id'].isin(booking_list)]
    print(df_booking_presciption_old.info(), "\n")
    print("Shape of data: ", df_booking_presciption_old.shape)

In [ ]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1:
    df_booking_presciption = pd.concat([df_booking_presciption, df_booking_presciption_old], axis=0)
    print("COMBINED: ")
    print(df_booking_presciption.info(), "\n")
    print("Shape of data: ", df_booking_presciption.shape)

### Summary Statistics

In [ ]:
print("Basic Data Summary")
df_booking_presciption.describe().transpose()

Basic Data Summary


,count,mean,std,min,25%,50%,75%,max
prescription_quantity,20241.0,16.386641,25.901118,1.0,5.0,10.0,20.0,1080.0


In [ ]:
print("Unique value count for each column")
count_unique(df_booking_presciption)

Unique value count for each column


,name,uniquecount
0,booking_id,8125
1,MDMedication_ID,406
2,MDMedication_name,405
3,is_fulfilled_externally,2
4,prescription_quantity,63
5,prescription_status,2


### Missing Value Checks

In [ ]:
print("number of null: ")
df_booking_presciption.isnull().sum()

number of null: 


booking_id                 0
MDMedication_ID            0
MDMedication_name          0
is_fulfilled_externally    0
prescription_quantity      0
prescription_status        0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Basic Cleaning

#### Remove comma, replace with semi-colon 

In [ ]:
df_booking_presciption.MDMedication_name = df_booking_presciption.MDMedication_name.str.replace(',', ';')

#### Check and remove duplicates

In [ ]:
print("number of duplicate to drop: ", df_booking_presciption.duplicated().sum())

number of duplicate to drop:  1


In [ ]:
if df_booking_presciption.duplicated().sum() > 0:
    df_booking_presciption = df_booking_presciption.drop_duplicates()

### Update Data Types

#### Change data types for categorical data

In [ ]:
df_booking_presciption['MDMedication_ID'] = df_booking_presciption['MDMedication_ID'].astype("category")

In [ ]:
df_booking_presciption['MDMedication_name'] = df_booking_presciption['MDMedication_name'].astype("category")

In [ ]:
df_booking_presciption['prescription_status'] = df_booking_presciption['prescription_status'].astype("category")

In [ ]:
df_booking_presciption.is_fulfilled_externally = df_booking_presciption.is_fulfilled_externally.replace({True: "True", False: "False"})
df_booking_presciption.is_fulfilled_externally = df_booking_presciption.is_fulfilled_externally.astype("category")

In [ ]:
df_booking_presciption.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20240 entries, 0 to 20240
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   booking_id               20240 non-null  object  
 1   MDMedication_ID          20240 non-null  category
 2   MDMedication_name        20240 non-null  category
 3   is_fulfilled_externally  20240 non-null  category
 4   prescription_quantity    20240 non-null  int64   
 5   prescription_status      20240 non-null  category
dtypes: category(4), int64(1), object(1)
memory usage: 631.9+ KB


### Check OntoOne Mapping

In [ ]:
g = df_booking_presciption[['MDMedication_ID', 'MDMedication_name']].groupby('MDMedication_name')
counts = g.transform(lambda x: len(x.unique()))

In [ ]:
df_booking_presciption["count_map"] = counts

In [ ]:
print("These are the records with more than one mapping for name vs code, please verify:")

These are the records with more than one mapping for name vs code, please verify:


In [ ]:
pd.set_option('display.max_colwidth', None)
df_booking_presciption.loc[df_booking_presciption["count_map"]>1,:]

,booking_id,MDMedication_ID,MDMedication_name,is_fulfilled_externally,prescription_quantity,prescription_status,count_map
146,FF983540-1C9A-4B34-88D8-43D82F83FF09,30F501C7-6DF8-469A-A691-03306015AB55,Herbesser R (Diltiazem) 100mg Cap 1's,True,180,NOT PURCHASED,2
147,360B7834-4AB8-46F2-BB58-6478411F23F7,30F501C7-6DF8-469A-A691-03306015AB55,Herbesser R (Diltiazem) 100mg Cap 1's,True,180,NOT PURCHASED,2
148,7B9504F5-D630-4A0F-88DB-8904FF6D3363,30F501C7-6DF8-469A-A691-03306015AB55,Herbesser R (Diltiazem) 100mg Cap 1's,True,180,NOT PURCHASED,2
149,71B937E4-FB02-4098-9D37-C1C356640427,30F501C7-6DF8-469A-A691-03306015AB55,Herbesser R (Diltiazem) 100mg Cap 1's,True,180,NOT PURCHASED,2
3558,1899D7FD-C566-4B15-9900-1675131A9F79,0B6BA838-5BDD-4E64-B099-31B1B46D91D4,Herbesser R (Diltiazem) 100mg Cap 1's,True,120,NOT PURCHASED,2


<span class="mark">IMPORTANT: 
Please verify data if above shows that there is duplicate mapping.</span> 

#### Remove Redundant field

In [ ]:
df_booking_presciption.drop(["count_map"], inplace = True, axis = 1)

In [ ]:
print("Unique value count for each column")
count_unique(df_booking_presciption)

Unique value count for each column


,name,uniquecount
0,booking_id,8125
1,MDMedication_ID,406
2,MDMedication_name,405
3,is_fulfilled_externally,2
4,prescription_quantity,63
5,prescription_status,2


### Data Aggregation

#### Check Unique Medication ID

In [ ]:
print(df_booking_presciption["MDMedication_ID"].nunique())

406


#### Check Unique Medication Name

In [ ]:
print(df_booking_presciption["MDMedication_name"].nunique())

405


#### Check Unique prescription_quantity

In [ ]:
print(df_booking_presciption["prescription_quantity"].nunique())

63


#### Roll up

The purpose here is to roll up all the diagnose of the same booking into one roll, while retaining all diagnosis info, by converting the information into a list.

In [ ]:
# get a dict of unique ID from booking  
booking_ID_list = set(df_booking_presciption["booking_id"])

In [ ]:
# initiate df to keep repeated medication for prescription
col_list = df_booking_presciption.columns.tolist()
df_repeat = pd.DataFrame(columns= col_list) 

In [ ]:
medID_dup_list = pd.DataFrame(columns=["booking_id","medicationID_list", "count_medication", "count_repeatmed"])
# to get a list of medication ID for each booking
count_repeat_overall = 0
for x in booking_ID_list:
    count_repeat_booking = 1
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    for index, row in temp_df.iterrows():
        if row["MDMedication_ID"] is not np.nan:
            if (row["MDMedication_ID"] not in temp_list):
                temp_list.append(row["MDMedication_ID"])
            else:
                count_repeat_booking += 1
                count_repeat_overall += 1
                df_repeat = df_repeat.append(row)
                #print("Repeated: ", str(row), "\n")
             
    medID_dup_list.at[x, "booking_id"] = x
    medID_dup_list.at[x, "medicationID_list"] = temp_list
    medID_dup_list.at[x, "count_medication"] = int(len(temp_list))
    medID_dup_list.at[x, "count_repeatmed"] = int(count_repeat_booking)
print("TOTAL REPEATED MED ROWS: ", str(count_repeat_overall))

TOTAL REPEATED MED ROWS:  34


In [ ]:
medName_dup_list = pd.DataFrame(columns=["booking_id","medicationName_list", "count_medName"])
# to get a list of medication ID for each booking
for x in booking_ID_list:
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDMedication_name"] is not np.nan:
            if (row["MDMedication_name"] not in temp_list):
                temp_list.append(row["MDMedication_name"])
             
    medName_dup_list.at[x, "booking_id"] = x
    medName_dup_list.at[x, "medicationName_list"] = temp_list
    medName_dup_list.at[x, "count_medName"] = int(len(temp_list))

In [ ]:
medquantity_dup_list = pd.DataFrame(columns=["booking_id","medQuantity_list", "count_medQuantity"])
# to get a list of medication ID for each booking
for x in booking_ID_list:
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    check_list = []
    for index, row in temp_df.iterrows():
        if row["prescription_quantity"] is not np.nan:
            if (row["MDMedication_name"] not in check_list):
                check_list.append(row["MDMedication_name"])
                temp_list.append(row["prescription_quantity"])
    medquantity_dup_list.at[x, "booking_id"] = x
    medquantity_dup_list.at[x, "medQuantity_list"] = temp_list
    medquantity_dup_list.at[x, "count_medQuantity"] = int(len(temp_list))

In [ ]:
medAndQuan_dup_list = pd.DataFrame(columns=["booking_id","medAndQuan_list", "medNQuanCount"])
# to get a list of medication ID for each booking
for x in booking_ID_list:
    temp_df = df_booking_presciption[df_booking_presciption["booking_id"] == x]
    temp_list = []
    check_list = []
    for index, row in temp_df.iterrows():
        if row["prescription_quantity"] is not np.nan:
            if (row["MDMedication_name"] not in check_list):
                check_list.append(row["MDMedication_name"])
                temp_list.append(str(row["MDMedication_name"]) + "|||" + str(row["prescription_quantity"]))
             
    medAndQuan_dup_list.at[x, "booking_id"] = x
    medAndQuan_dup_list.at[x, "medAndQuan_list"] = temp_list
    medAndQuan_dup_list.at[x, "medNQuanCount"] = int(len(temp_list))

In [ ]:
# Perform an inner join between the prescription name and code temp tables
combine_data_med1 = pd.merge(medID_dup_list, medName_dup_list, how="inner", on=["booking_id"])

In [ ]:
# Perform an inner join between the above and quantity temp tables
combine_data_med2 = pd.merge(combine_data_med1, medquantity_dup_list, how="inner", on=["booking_id"])

In [ ]:
# Perform an inner join between the above and medication&quantity pairs temp tables
combine_data_med3 = pd.merge(combine_data_med2, medAndQuan_dup_list, how="inner", on=["booking_id"])

#### Finalise Prep

In [ ]:
col_list = ["booking_id","medicationID_list","medicationName_list","medQuantity_list", "medAndQuan_list", 
            "count_medication", "count_repeatmed"]

In [ ]:
final_df_booking_presciption = combine_data_med3[col_list]

In [ ]:
final_df_booking_presciption.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8125 entries, 0 to 8124
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           8125 non-null   object
 1   medicationID_list    8125 non-null   object
 2   medicationName_list  8125 non-null   object
 3   medQuantity_list     8125 non-null   object
 4   medAndQuan_list      8125 non-null   object
 5   count_medication     8125 non-null   object
 6   count_repeatmed      8125 non-null   object
dtypes: object(7)
memory usage: 507.8+ KB


In [ ]:
final_df_booking_presciption.head(3)

,booking_id,medicationID_list,medicationName_list,medQuantity_list,medAndQuan_list,count_medication,count_repeatmed
0,2532BFF4-5C2F-4256-A738-EB38EB1A6139,"[960B1495-908B-48DA-A134-94A2A328B1D1, 2B5F01AA-351A-407D-AFE8-FF38EE378BA5]","[Naproxen Sodium 275mg tab, Paracetamol 500mg tab]","[30, 30]","[Naproxen Sodium 275mg tab|||30, Paracetamol 500mg tab|||30]",2,1
1,E6952025-2750-4123-8963-4D5E9D1F0DB2,"[3A89CEAA-D15E-4371-9CBF-80492596E0F1, 960B1495-908B-48DA-A134-94A2A328B1D1]","[Famotidine 20mg tab (Sunpepcin), Naproxen Sodium 275mg tab]","[10, 20]","[Famotidine 20mg tab (Sunpepcin)|||10, Naproxen Sodium 275mg tab|||20]",2,1
2,8F396092-C568-4F35-9A60-22C2471122F7,"[9E8EA48E-3980-41E9-85A2-70CFB35B9F8F, C5A0B481-BC40-4EF5-AC5F-9E2657A6FADE]","[Etoricoxib 90mg tab (Arcoxia), Domperidone 10mg tab]","[5, 10]","[Etoricoxib 90mg tab (Arcoxia)|||5, Domperidone 10mg tab|||10]",2,1


#### Save intermediate Output

In [ ]:
# Export data 
final_df_booking_presciption.to_pickle("step1_booking_prescription.pkl")

<span class="burk">**COMPLETE STEP1 FOR BOOKING PRESCRIPTION**</span>

## Step1c - Prep & Clean Booking Symptoms

### Import booking symptoms data

In [ ]:
# import new booking symptoms
print("Input: ", FILE_SYMPTOMS, "\n") 
df_booking_symptoms = pd.read_csv(FILE_SYMPTOMS)
df_booking_symptoms = df_booking_symptoms[df_booking_symptoms['booking_id'].isin(booking_list)]
print(df_booking_symptoms.info(), "\n") 
print("Shape of data: ", df_booking_symptoms.shape)

Input:  tbl_Booking_Symptoms.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18096 entries, 0 to 18095
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   booking_id      18096 non-null  object
 1   MDSymptom_id    18096 non-null  object
 2   MDSymptom_name  18096 non-null  object
dtypes: object(3)
memory usage: 565.5+ KB
None 

Shape of data:  (18096, 3)


In [ ]:
# import old booking symptoms
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_SYMPTOMS_OLD, "\n") 
    df_booking_symptoms_old = pd.read_csv(FILE_SYMPTOMS_OLD)
    df_booking_symptoms_old = df_booking_symptoms_old[df_booking_symptoms_old['booking_id'].isin(booking_list)]
    print(df_booking_symptoms_old.info(), "\n")
    print("Shape of data: ", df_booking_symptoms_old.shape)

In [ ]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1:
    df_booking_symptoms = pd.concat([df_booking_symptoms, df_booking_symptoms_old], axis=0)
    print("COMBINED: ")
    print(df_booking_symptoms.info(), "\n")
    print("Shape of data: ", df_booking_symptoms.shape)

### Summary Statistics

In [ ]:
print("Basic Data Summary")
df_booking_symptoms.describe().transpose()

Basic Data Summary


,count,unique,top,freq
booking_id,18096,9066,541A4558-33C5-42DF-9E4F-90A06B63C921,10
MDSymptom_id,18096,21,A518BD9F-95E0-411F-B058-F4DEA6AD1884,3976
MDSymptom_name,18096,21,others,3976


In [ ]:
print("Unique value count for each column")
count_unique(df_booking_symptoms)

Unique value count for each column


,name,uniquecount
0,booking_id,9066
1,MDSymptom_id,21
2,MDSymptom_name,21


### Missing Value Checks

In [ ]:
print("number of null: ")
df_booking_symptoms.isnull().sum()

number of null: 


booking_id        0
MDSymptom_id      0
MDSymptom_name    0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Basic Cleaning

#### Change text to all lower case

In [ ]:
df_booking_symptoms['MDSymptom_name'] = df_booking_symptoms['MDSymptom_name'].str.lower()

#### Remove comma, replace with semi-colon 

In [ ]:
df_booking_symptoms.MDSymptom_name = df_booking_symptoms.MDSymptom_name.str.replace(',', ';')

#### Check and remove duplicates

In [ ]:
print("number of duplicate to drop: ", df_booking_symptoms.duplicated().sum())

number of duplicate to drop:  0


In [ ]:
# to drop duplicates 
if df_booking_symptoms.duplicated().sum() > 0:
    df_booking_symptoms = df_booking_symptoms.drop_duplicates()

### Update Data Types

#### Change data types for categorical data

In [ ]:
df_booking_symptoms['MDSymptom_id'] = df_booking_symptoms['MDSymptom_id'].astype("category")

In [ ]:
df_booking_symptoms['MDSymptom_name'] = df_booking_symptoms['MDSymptom_name'].astype("category")

In [ ]:
df_booking_symptoms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18096 entries, 0 to 18095
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   booking_id      18096 non-null  object  
 1   MDSymptom_id    18096 non-null  category
 2   MDSymptom_name  18096 non-null  category
dtypes: category(2), object(1)
memory usage: 319.5+ KB


In [ ]:
print("List of Chronic Disease:")
df_booking_symptoms['MDSymptom_name'].unique().tolist()

List of Chronic Disease:


['fever',
 'others',
 'sore throat',
 'flu',
 'cough',
 'cold',
 'vomiting',
 'medical referral',
 'diarrhoea',
 'headache',
 'giddiness',
 'rash',
 'cold sores',
 'chronic medicine top-up',
 'discuss lab results',
 'chronic illness management',
 'runny nose',
 'abdominal pain',
 'dysmenorrhoea',
 'vertigo',
 'insect bite']

### Check OntoOne Mapping

#### Check by ID

In [ ]:
g = df_booking_symptoms[['MDSymptom_id', 'MDSymptom_name']].groupby('MDSymptom_id')
counts = g.transform(lambda x: len(x.unique()))
(counts>1).sum()

MDSymptom_name    0
dtype: int64

#### Check by Name

In [ ]:
g2 = df_booking_symptoms[['MDSymptom_id', 'MDSymptom_name']].groupby('MDSymptom_name')
counts2 = g2.transform(lambda x: len(x.unique()))
(counts2>1).sum()

MDSymptom_id    0
dtype: int64

<span class="mark">IMPORTANT: 
Please verify data if above shows that there is duplicate mapping.</span> 

### Data Aggregation

#### Check unique symptoms ID

In [ ]:
print(df_booking_symptoms["MDSymptom_id"].nunique())

21


#### Check unique symptoms Name

In [ ]:
print(df_booking_symptoms["MDSymptom_name"].nunique())

21


In [ ]:
print(df_booking_symptoms["MDSymptom_name"].value_counts())

others                        3976
headache                      2337
giddiness                     2240
diarrhoea                     1402
vomiting                      1329
sore throat                    996
flu                            990
cold                           980
cough                          978
rash                           589
cold sores                     571
fever                          557
chronic medicine top-up        419
medical referral               415
discuss lab results            255
chronic illness management      57
dysmenorrhoea                    1
insect bite                      1
runny nose                       1
vertigo                          1
abdominal pain                   1
Name: MDSymptom_name, dtype: int64


In [ ]:
print(df_booking_symptoms["MDSymptom_name"].value_counts(normalize=True)*100)

others                        21.971706
headache                      12.914456
giddiness                     12.378426
diarrhoea                      7.747569
vomiting                       7.344164
sore throat                    5.503979
flu                            5.470822
cold                           5.415561
cough                          5.404509
rash                           3.254863
cold sores                     3.155393
fever                          3.078028
chronic medicine top-up        2.315429
medical referral               2.293324
discuss lab results            1.409151
chronic illness management     0.314987
dysmenorrhoea                  0.005526
insect bite                    0.005526
runny nose                     0.005526
vertigo                        0.005526
abdominal pain                 0.005526
Name: MDSymptom_name, dtype: float64


#### Roll up

In [ ]:
# get a dict of unique ID from booking  
booking_ID_list = set(df_booking_symptoms["booking_id"])

In [ ]:
smyID_dup_list = pd.DataFrame(columns=["booking_id","SymptomID_list", "count_SymID"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_symptoms[df_booking_symptoms["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDSymptom_id"] is not np.nan:
            if (row["MDSymptom_id"] not in tmp_list):
                tmp_list.append(row["MDSymptom_id"])
             
    smyID_dup_list.at[x, "booking_id"] = x
    smyID_dup_list.at[x, "SymptomID_list"] = tmp_list
    smyID_dup_list.at[x, "count_SymID"] = int(len(tmp_list))

In [ ]:
smyName_dup_list = pd.DataFrame(columns=["booking_id","SymptomName_list", "count_SymName"])
# to get a list of diagnosis name for each booking
for x in booking_ID_list:
    temp_df = df_booking_symptoms[df_booking_symptoms["booking_id"] == x]
    tmp_list = []
    for index, row in temp_df.iterrows():
        #print(row["MDDiagnosis_name"]) 
        if row["MDSymptom_name"] is not np.nan:
            if (row["MDSymptom_name"] not in tmp_list):
                tmp_list.append(row["MDSymptom_name"])
             
    smyName_dup_list.at[x, "booking_id"] = x
    smyName_dup_list.at[x, "SymptomName_list"] = tmp_list
    smyName_dup_list.at[x, "count_SymName"] = int(len(tmp_list))

### Finalise Prep

In [ ]:
# Perform an inner join between the diagnosis name and code temp tables
combine_data_symp = pd.merge(smyID_dup_list, smyName_dup_list, how="inner", on=["booking_id"])

In [ ]:
final_df_booking_symptoms = combine_data_symp[["booking_id","SymptomID_list","SymptomName_list",
                              "count_SymID"]]

In [ ]:
final_df_booking_symptoms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9066 entries, 0 to 9065
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        9066 non-null   object
 1   SymptomID_list    9066 non-null   object
 2   SymptomName_list  9066 non-null   object
 3   count_SymID       9066 non-null   object
dtypes: object(4)
memory usage: 354.1+ KB


In [ ]:
final_df_booking_symptoms.shape

(9066, 4)

In [ ]:
final_df_booking_symptoms.head(3)

,booking_id,SymptomID_list,SymptomName_list,count_SymID
0,CAA7BA7B-B12F-4E1E-9A7A-2284A72BF9CB,"[C3AF5B1F-BD74-4700-A1E7-1958B32C80BD, B8F546F4-9C2A-4E31-8ED5-DB3187FA4351]","[vomiting, diarrhoea]",2
1,2532BFF4-5C2F-4256-A738-EB38EB1A6139,[A518BD9F-95E0-411F-B058-F4DEA6AD1884],[others],1
2,E6952025-2750-4123-8963-4D5E9D1F0DB2,"[6E7F72BF-3C7B-4F0D-B1D2-5E9FB9B5CE9B, 1B6BFF99-1553-487A-A85E-EBF552D50D23]","[headache, giddiness]",2


### Save intermediate Ouput

In [ ]:
# Export data 
final_df_booking_symptoms.to_pickle("step1_booking_symptoms.pkl")

<span class="burk">**COMPLETE STEP1 FOR BOOKING SYMPTOMS**</span>

<span class="burk">COMPLETE ALL STEP1</span>

## Step2 - Merge Data Tables 

Merge Strategy:

1. Combine Booking and Patient to become the Base
2. Combine Base + Diagnosis(Inner) + Prescription(Outer) + Symptoms(Outer)

Note: Only include booking with Diagnosis

Note: Lab data excluded

In [ ]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP1_DATAPREP = 0

### Import Prep Data Intermediate Data

In [ ]:
# Load data
if LOAD_INTERMEDIATE_STEP1_DATAPREP == 1:
    df_diagnosis = read_pickle("step1_booking_diagnosis.pkl")
    df_presciption = read_pickle("step1_booking_prescription.pkl")
    df_symptoms = read_pickle("step1_booking_symptoms.pkl")
    open_file = open("step1_booking_list.pkl", "rb")
    booking_list = pickle.load(open_file)
    open_file.close()
else:
    df_diagnosis = final_df_booking_diagnosis
    df_prescription = final_df_booking_presciption
    df_symptoms = final_df_booking_symptoms
    booking_list = booking_list

In [ ]:
print("Booking Diagnosis: ", df_diagnosis.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           10221 non-null  object
 1   diagnosis_code_list  10221 non-null  object
 2   diagnosis_list       10221 non-null  object
 3   chronic_list         10221 non-null  object
 4   count_diagnosis      10221 non-null  object
 5   count_chronic        10221 non-null  object
dtypes: object(6)
memory usage: 559.0+ KB
Booking Diagnosis:  None


In [ ]:
print("Booking Prescription:")
print(df_prescription.info())

Booking Prescription:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8125 entries, 0 to 8124
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   booking_id           8125 non-null   object
 1   medicationID_list    8125 non-null   object
 2   medicationName_list  8125 non-null   object
 3   medQuantity_list     8125 non-null   object
 4   medAndQuan_list      8125 non-null   object
 5   count_medication     8125 non-null   object
 6   count_repeatmed      8125 non-null   object
dtypes: object(7)
memory usage: 507.8+ KB
None


In [ ]:
print("Booking Symptoms:")
print(df_symptoms.info())

Booking Symptoms:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9066 entries, 0 to 9065
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   booking_id        9066 non-null   object
 1   SymptomID_list    9066 non-null   object
 2   SymptomName_list  9066 non-null   object
 3   count_SymID       9066 non-null   object
dtypes: object(4)
memory usage: 354.1+ KB
None


### Import Source Booking Data

In [ ]:
# import new booking 
print("Input: ", FILE_BOOKING, "\n") 
df_booking = pd.read_csv(FILE_BOOKING)
df_booking = df_booking[df_booking['booking_id'].isin(booking_list)]
print("Basic Data structure Info (NEW) ", df_booking.info(), "\n") 
print("Shape of data: ", df_booking.shape)

Input:  tbl_Booking.csv 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
dtypes: bool(2), int64(1), object(5)
memory usage: 578.9+ KB
Basic Data structure Info (NEW)  None 

Shape of data:  (10221, 8)


In [ ]:
# import old booking 
if TESTING==1 and MODEL_TYPE_MEMBER ==1:
    print("Input: ", FILE_BOOKING_OLD, "\n") 
    df_booking_old = pd.read_csv(FILE_BOOKING_OLD)
    df_booking_old = df_booking_old[df_booking_old['booking_id'].isin(booking_list)]
    print("Basic Data structure Info ", df_booking_old.info(), "\n")
    print("Shape of data: ", df_booking_old.shape)

In [ ]:
if TESTING ==1 and MODEL_TYPE_MEMBER ==1:
    df_booking = pd.concat([df_booking, df_booking_old], axis=0)
    print("Basic Data structure Info ", df_booking.info(), "\n")
    print("Shape of data: ", df_booking.shape)

In [ ]:
### Import Source Paitent Data
df_patient = pd.read_csv(FILE_PATIENT)
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   mem_id      5000 non-null   object
 1   gender      5000 non-null   object
 2   birth_year  5000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


### Merge Data

### Combine Booking and Patient Data

In [ ]:
combine_book_patient = pd.merge(df_booking, df_patient, how="inner", on=["mem_id"])

In [ ]:
combine_book_patient.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
 8   gender                     10221 non-null  object
 9   birth_year                 10221 non-null  int64 
dtypes: bool(2), int64(2), object(6)
memory usage: 738.6+ KB


In [ ]:
print("Unique value count for each column")
count_unique(combine_book_patient)

Unique value count for each column


,name,uniquecount
0,booking_id,10221
1,mem_id,5000
2,doctor_id,29
3,consult_waived_reason,7
4,is_mc_issued,2
5,mc_days,9
6,is_referral_letter_issued,2
7,booking_timestamp,10220
8,gender,3
9,birth_year,70


### Missing Value Checks

In [ ]:
print("number of null: ")
combine_book_patient.isnull().sum()

number of null: 


booking_id                   0
mem_id                       0
doctor_id                    0
consult_waived_reason        0
is_mc_issued                 0
mc_days                      0
is_referral_letter_issued    0
booking_timestamp            0
gender                       0
birth_year                   0
dtype: int64

<span class="mark">IMPORTANT: 
If there is missing value, pls handle and correct accordingly.</span>

### Check and remove duplicates

In [ ]:
print("number of duplicate to drop: ", combine_book_patient.duplicated().sum())

number of duplicate to drop:  0


In [ ]:
# to drop duplicates 
if combine_book_patient.duplicated().sum() > 0:
    combine_book_patient = combine_book_patient.drop_duplicates()

### Combine Base with Diagnoisis

In [ ]:
combine_base_diagnosis_inner = pd.merge(combine_book_patient, df_diagnosis, how="inner", on=["booking_id"])

In [ ]:
combine_base_diagnosis_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
 8   gender                     10221 non-null  object
 9   birth_year                 10221 non-null  int64 
 10  diagnosis_code_list        10221 non-null  object
 11  diagnosis_list             10221 non-null  object
 12  chronic_list               10221 non-null  object
 13  count_diagnosis            10221 non-null  object
 14  count_

### Combine (Base + Diagnosis)Inner + Prescription(Outer)

In [ ]:
combine_base_diagInner_presOuter = pd.merge(combine_base_diagnosis_inner, 
                                            df_prescription, how="outer", on=["booking_id"])

In [ ]:
combine_base_diagInner_presOuter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
 8   gender                     10221 non-null  object
 9   birth_year                 10221 non-null  int64 
 10  diagnosis_code_list        10221 non-null  object
 11  diagnosis_list             10221 non-null  object
 12  chronic_list               10221 non-null  object
 13  count_diagnosis            10221 non-null  object
 14  count_

### Combine (Base + Diagnosis)Inner + Presc(Outer) + Sym(Outer)

In [ ]:
combine_base_diagInner_presOuter_symOuter = pd.merge(combine_base_diagInner_presOuter, df_symptoms, 
                                                     how="outer", on=["booking_id"])

In [ ]:
combine_base_diagInner_presOuter_symOuter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
 8   gender                     10221 non-null  object
 9   birth_year                 10221 non-null  int64 
 10  diagnosis_code_list        10221 non-null  object
 11  diagnosis_list             10221 non-null  object
 12  chronic_list               10221 non-null  object
 13  count_diagnosis            10221 non-null  object
 14  count_

In [ ]:
final_df = combine_base_diagInner_presOuter_symOuter

### Save Intermediate Output

In [ ]:
# Export data 
final_df.to_pickle("step2_merge.pkl")

<span class="burk">COMPLETE STEP 2</span>

## Step3 - Intro Cast & Transpose for Member

To summarize the following categories into columns:
* Chronic
* Diagnosis Code
* Symptoms

### Import Merged Data from Step2

In [ ]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP2_DATAPREP = 0

In [ ]:
# Load data
if LOAD_INTERMEDIATE_STEP2_DATAPREP == 1:
    df_final_2 = read_pickle("step2_merge.pkl")
else:
    df_final_2 = final_df

In [ ]:
print(df_final_2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   booking_id                 10221 non-null  object
 1   mem_id                     10221 non-null  object
 2   doctor_id                  10221 non-null  object
 3   consult_waived_reason      10221 non-null  object
 4   is_mc_issued               10221 non-null  bool  
 5   mc_days                    10221 non-null  int64 
 6   is_referral_letter_issued  10221 non-null  bool  
 7   booking_timestamp          10221 non-null  object
 8   gender                     10221 non-null  object
 9   birth_year                 10221 non-null  int64 
 10  diagnosis_code_list        10221 non-null  object
 11  diagnosis_list             10221 non-null  object
 12  chronic_list               10221 non-null  object
 13  count_diagnosis            10221 non-null  object
 14  count_

## Step3a - Cast & Transpose for Chronic

### Check NA for Chronic

In [ ]:
df_final_2['chronic_list'] = df_final_2['chronic_list'].fillna("N").apply(list)

### Process Chronic String

In [ ]:
df_final_2["chronic_list_str"] = (df_final_2["chronic_list"].apply(lambda x: ",".join(map(str, x))))

In [ ]:
#process string
df_final_2['chronic_list_str'] = df_final_2['chronic_list_str'].replace("","NoneChronic")

In [ ]:
# To check
df_final_2['chronic_list_str'].head(20)

0           NoneChronic
1           NoneChronic
2           NoneChronic
3           NoneChronic
4     Atopic dermatitis
5           NoneChronic
6           NoneChronic
7           NoneChronic
8           NoneChronic
9              Migraine
10          NoneChronic
11          NoneChronic
12          NoneChronic
13    Atopic dermatitis
14          NoneChronic
15          NoneChronic
16          NoneChronic
17          NoneChronic
18          NoneChronic
19          NoneChronic
Name: chronic_list_str, dtype: object

### Create Chronic Dict

In [ ]:
chronic = df_final_2['chronic_list_str']

In [ ]:
#create a dictionary of symptoms with frequency
chronic_dict = {}
for i in chronic:
    li = i.split(",")
    for k in li:
        if k in chronic_dict:
            chronic_dict[k] += 1
        else:
            chronic_dict[k] = 1

print("There are", len(chronic_dict), 
      "unique chronic ! Inclucing all of them as features will be detremental to performance")

There are 11 unique chronic ! Inclucing all of them as features will be detremental to performance


In [ ]:
chronic_dict

{'NoneChronic': 8734,
 'Atopic dermatitis': 195,
 'Migraine': 190,
 'Vasomotor and allergic rhinitis': 377,
 'Gastro-oesophageal reflux disease': 200,
 'Hypertension': 241,
 'Asthma': 148,
 'Hyperlipidaemia': 176,
 'Thyroid': 44,
 'Diabetes': 57,
 'Gout': 125}

### Create Chronic Summary DF

In [ ]:
chronic_df = pd.DataFrame(columns=chronic_dict.keys(), index=range(len(chronic)))

In [ ]:
for chronic_i in (chronic_dict): 
    chronic_df[chronic_i] = chronic.str.contains(chronic_i)

In [ ]:
chronic_df = chronic_df.add_prefix("chronic_")

In [ ]:
new_df_chronic_summarize = pd.concat([df_final_2["booking_id"], chronic_df], axis=1)

In [ ]:
new_df_chronic_summarize.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 12 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   booking_id                                 10221 non-null  object
 1   chronic_NoneChronic                        10221 non-null  bool  
 2   chronic_Atopic dermatitis                  10221 non-null  bool  
 3   chronic_Migraine                           10221 non-null  bool  
 4   chronic_Vasomotor and allergic rhinitis    10221 non-null  bool  
 5   chronic_Gastro-oesophageal reflux disease  10221 non-null  bool  
 6   chronic_Hypertension                       10221 non-null  bool  
 7   chronic_Asthma                             10221 non-null  bool  
 8   chronic_Hyperlipidaemia                    10221 non-null  bool  
 9   chronic_Thyroid                            10221 non-null  bool  
 10  chronic_Diabetes                  

In [ ]:
new_df_chronic_summarize.shape

(10221, 12)

In [ ]:
# Tocheck
new_df_chronic_summarize["chronic_NoneChronic"].sum()

8734

In [ ]:
# Tocheck
new_df_chronic_summarize["chronic_Gout"].sum()

125

In [ ]:
new_df_chronic_summarize.head().transpose()

,0,1,2,3,4
booking_id,23800D32-7520-4D21-83A5-80A378443767,8D2D1C67-CB20-4453-A70F-503DFCF506F8,0EB28D3C-F967-476F-BA26-EC89C53F8297,64005993-7B8D-4AC6-B184-2BC260995C36,04DF2127-C262-493C-A7BE-1C5F33F8B378
chronic_NoneChronic,True,True,True,True,False
chronic_Atopic dermatitis,False,False,False,False,True
chronic_Migraine,False,False,False,False,False
chronic_Vasomotor and allergic rhinitis,False,False,False,False,False
chronic_Gastro-oesophageal reflux disease,False,False,False,False,False
chronic_Hypertension,False,False,False,False,False
chronic_Asthma,False,False,False,False,False
chronic_Hyperlipidaemia,False,False,False,False,False
chronic_Thyroid,False,False,False,False,False


## Step3b - Cast and Transpose for Diagnosis Code

### Check NA for Diagnosis Code

In [ ]:
df_final_2['diagnosis_code_list'] = df_final_2['diagnosis_code_list'].fillna("N").apply(list)

### Process Diagnosis Code String

In [ ]:
df_final_2["diagnosis_code_list_str"] = (df_final_2["diagnosis_code_list"].apply(lambda x: ",".join(map(str, x))))

In [ ]:
# Remove the dot to reduce the category
diagnosis_code = df_final_2['diagnosis_code_list_str']
diagnosis_code = diagnosis_code.apply(lambda x: re.sub('\.\d+', '', x))

### Create Diagnosis Code Dict

In [ ]:
#create a dictionary of symptoms with frequency
diagnosis_code_dict = {}
temp=[]
for i in diagnosis_code:
    li = i.split(",")
    for k in li:
        k = re.sub('\.\d+', '', k)
        if k in temp:
            diagnosis_code_dict[k] += 1
        else:
            diagnosis_code_dict[k] = 1
            temp.append(k)

print("There are", len(diagnosis_code_dict))

There are 349


### (Optional) Set Min Count to include for Features (for model training)

Removed diagnosis that happens less than 10 times, 
to reduce the features, that is equivalent to 0.1% of total number of obervation

In [ ]:
# remove min value (in case required to cut)
selected_diag = {key:val for key, val in diagnosis_code_dict.items() if val > MIN_VAL_DIAG}

### Create Diagnosis Code Summary DF

In [ ]:
diagnosis_df = pd.DataFrame(columns=selected_diag.keys(), index=range(len(diagnosis_code)))

In [ ]:
for diag_i in (selected_diag): 
    diagnosis_df[diag_i] = diagnosis_code.apply(lambda x: str(diag_i) in str(x))

In [ ]:
diagnosis_df = diagnosis_df.add_prefix("diag_")

In [ ]:
diagnosis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10221 entries, 0 to 10220
Data columns (total 99 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   diag_H57  10221 non-null  bool 
 1   diag_M23  10221 non-null  bool 
 2   diag_T88  10221 non-null  bool 
 3   diag_L20  10221 non-null  bool 
 4   diag_R51  10221 non-null  bool 
 5   diag_N94  10221 non-null  bool 
 6   diag_G44  10221 non-null  bool 
 7   diag_M79  10221 non-null  bool 
 8   diag_J02  10221 non-null  bool 
 9   diag_J06  10221 non-null  bool 
 10  diag_G43  10221 non-null  bool 
 11  diag_R21  10221 non-null  bool 
 12  diag_M62  10221 non-null  bool 
 13  diag_A09  10221 non-null  bool 
 14  diag_J30  10221 non-null  bool 
 15  diag_L30  10221 non-null  bool 
 16  diag_T78  10221 non-null  bool 
 17  diag_U07  10221 non-null  bool 
 18  diag_K29  10221 non-null  bool 
 19  diag_B35  10221 non-null  bool 
 20  diag_K30  10221 non-null  bool 
 21  diag_H81  10221 non-null  bool 
 22

In [ ]:
new_df_diag_summarize = pd.concat([df_final_2["booking_id"], diagnosis_df], axis=1)

In [ ]:
new_df_diag_summarize.shape

(10221, 100)

## Step3c - Cast & Transpose for Symptoms

### Check NA for Symptoms 

In [ ]:
df_final_2['SymptomName_list'] = df_final_2['SymptomName_list'].fillna("N").apply(list)

### Process Symptoms String

In [ ]:
df_final_2["SymptomName_list_str"] = (df_final_2["SymptomName_list"].apply(lambda x: ",".join(map(str, x))))

In [ ]:
#process string
symp = df_final_2['SymptomName_list_str']

### Create Symptoms Dict

In [ ]:
#create a dictionary of symptoms with frequency
symp_dict = {}
for i in symp:
    li = i.split(",")
    for k in li:
        if k in symp_dict:
            symp_dict[k] += 1
        else:
            symp_dict[k] = 1

print("There are", len(symp_dict), "unique symptoms ! \
      Inclucing all of them as features will be detremental to performance, do check")

There are 22 unique symptoms !       Inclucing all of them as features will be detremental to performance, do check


In [ ]:
symp_dict

{'others': 3976,
 'medical referral': 415,
 'headache': 2337,
 'giddiness': 2240,
 'fever': 557,
 'N': 1155,
 'cold sores': 571,
 'rash': 589,
 'sore throat': 996,
 'flu': 990,
 'cough': 978,
 'cold': 980,
 'vomiting': 1329,
 'diarrhoea': 1402,
 'chronic medicine top-up': 419,
 'chronic illness management': 57,
 'discuss lab results': 255,
 'runny nose': 1,
 'vertigo': 1,
 'dysmenorrhoea': 1,
 'abdominal pain': 1,
 'insect bite': 1}

### (Optional) Set Min Count to include for Features (for modeling)

Removed symptoms that happens less than 10 times, to reduce the features.

In [ ]:
# remove min value (in case required to cut)
selected_symptoms = {key:val for key, val in symp_dict.items() if val > MIN_VAL_SYM}

### Create Symptoms Summary DF

In [ ]:
symptom_df = pd.DataFrame(columns=selected_symptoms.keys(), index=range(len(symp)))

In [ ]:
for symptom in (selected_symptoms): 
    symptom_df[symptom] = symp.str.contains(symptom)

In [ ]:
symptom_df = symptom_df.add_prefix("symp_")

In [ ]:
new_df_symp_summarize = pd.concat([df_final_2["booking_id"], symptom_df], axis=1)

In [ ]:
new_df_symp_summarize.shape

(10221, 18)

## Step4 - Merge Summarized Data

In [ ]:
# drop unnec fields
df_final_2.drop(["chronic_list_str", "diagnosis_code_list_str", "SymptomName_list_str"], inplace = True, axis = 1)

In [ ]:
combine_expandDiag = pd.merge(df_final_2, new_df_diag_summarize, 
                              how="inner", on=["booking_id"])

In [ ]:
combine_expandDiagChronic = pd.merge(combine_expandDiag, new_df_chronic_summarize, 
                                     how="inner", on=["booking_id"])

In [ ]:
combine_expandDiagChronicSym = pd.merge(combine_expandDiagChronic, new_df_symp_summarize, 
                                        how="inner", on=["booking_id"])

In [ ]:
combine_expandDiagChronicSym.shape

(10221, 151)

### Save intermediate Output

In [ ]:
# Export data 
combine_expandDiagChronicSym.to_pickle("step4_merge_summarized.pkl")

<span class="burk">COMPLETE STEP 4</span>

## Step5 - Finalise Booking Data

### Import Step 4 merged data

In [ ]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP4_DATAPREP = 0

In [ ]:
# Load data
if LOAD_INTERMEDIATE_STEP4_DATAPREP == 1:
    df_final_3 = read_pickle("step4_merge_summarized.pkl")
else:
    df_final_3 = combine_expandDiagChronicSym

### Create new variable - Target

In [ ]:
df_final_3["Target_chronic"] = df_final_3["count_chronic"].apply(lambda x: 1 if x>0 else 0)

### Change Category Data Type

In [ ]:
# change data types for categorical data
print("Change Data Type to Category")

Change Data Type to Category


In [ ]:
cat_list = ["doctor_id", "consult_waived_reason", "gender"]

In [ ]:
for col in cat_list:
    df_final_3[col] = df_final_3[col].astype("category")

### Create New Numeric variable - Age

In [ ]:
today = date.today()

In [ ]:
df_final_3["age_yr"] = today.year - df_final_3["birth_year"]

### Change Category Data Type

In [ ]:
num_list = ['mc_days',"age_yr",
 'count_diagnosis',
 'count_chronic',
 'count_medication',
 'count_repeatmed',
 'count_SymID'
 ]

In [ ]:
for col in num_list:
    df_final_3[col] = df_final_3[col].astype("float")

### Change Boolean to Binary Dummy

In [ ]:
filter_col_diag = [col for col in df_final_3 if col.startswith('diag_')]

In [ ]:
filter_col_chronic = [col for col in df_final_3 if col.startswith('chronic_')]

In [ ]:
filter_col_symp = [col for col in df_final_3 if col.startswith('symp_')]

In [ ]:
fitler_col_other_bool =  ["is_mc_issued", "is_referral_letter_issued", "Target_chronic"]

In [ ]:
boolean_list = fitler_col_other_bool + filter_col_diag + filter_col_chronic + filter_col_symp

In [ ]:
for col in boolean_list:
    df_final_3[col] = df_final_3[col].apply(lambda x: 1 if x==True else 0)

### Change data types to date time 

In [ ]:
# change data types to date time
df_final_3["booking_timestamp"] = pd.to_datetime(df_final_3["booking_timestamp"])

### Create New Variable Consult Hour

In [ ]:
# check latest time
df_final_3["consult_hr"] = df_final_3["booking_timestamp"].dt.hour

In [ ]:
df_final_3["consult_hr"] = df_final_3["consult_hr"].astype("category")

### Impute Missing Data

In [ ]:
miss_list = ["count_medication", "count_repeatmed", "count_SymID"]

In [ ]:
for col in miss_list:
    df_final_3[col] = df_final_3[col].fillna(0)

### Impute with "NA"

In [ ]:
df_final_3["consult_waived_reason"] = df_final_3["consult_waived_reason"].apply(lambda x: "NA" if x =="-" else x)

In [ ]:
df_final_3["consult_waived_reason"].value_counts()

NA                            9712
FOLLOW UP                      195
HEALTH SCREENING REVIEW        176
OTHERS                          75
UNSUITABLE FOR TELECONSULT      45
WHITECOAT FAMILY                15
REFERRAL                         3
Name: consult_waived_reason, dtype: int64

### Export Final at Bookling Level

In [ ]:
print("Shape of data: ", df_final_3.shape)

Shape of data:  (10221, 154)


In [ ]:
df_final_3.info("deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 154 columns):
 #    Column                                     Dtype         
---   ------                                     -----         
 0    booking_id                                 object        
 1    mem_id                                     object        
 2    doctor_id                                  category      
 3    consult_waived_reason                      category      
 4    is_mc_issued                               int64         
 5    mc_days                                    float64       
 6    is_referral_letter_issued                  int64         
 7    booking_timestamp                          datetime64[ns]
 8    gender                                     category      
 9    birth_year                                 int64         
 10   diagnosis_code_list                        object        
 11   diagnosis_list                             object   

In [ ]:
pd.set_option('display.max_rows', None)
df_final_3.head().transpose()

,0,1,2,3,4
booking_id,23800D32-7520-4D21-83A5-80A378443767,8D2D1C67-CB20-4453-A70F-503DFCF506F8,0EB28D3C-F967-476F-BA26-EC89C53F8297,64005993-7B8D-4AC6-B184-2BC260995C36,04DF2127-C262-493C-A7BE-1C5F33F8B378
mem_id,3499A82B-4260-4447-9CDC-152262D968E5,3499A82B-4260-4447-9CDC-152262D968E5,B86F2734-B200-4B16-BF6B-7B873EA651A4,B86F2734-B200-4B16-BF6B-7B873EA651A4,B86F2734-B200-4B16-BF6B-7B873EA651A4
doctor_id,B045D798-DC64-4CA9-9FC4-28691E82EB72,77EFF8BB-D181-4BFB-90A4-75FA494F4BBB,EB022DC9-CBDB-4E11-950B-ECBB2287E2C3,EB022DC9-CBDB-4E11-950B-ECBB2287E2C3,7C2FDBF3-2942-47AD-AE19-ECC6F4E115E9
consult_waived_reason,NA,NA,NA,NA,NA
is_mc_issued,0,0,1,1,1
mc_days,0.0,0.0,1.0,1.0,1.0
is_referral_letter_issued,0,1,0,0,0
booking_timestamp,2021-05-07 12:37:02,2021-04-26 16:44:15,2021-08-23 19:32:00,2021-08-01 16:33:53,2021-02-22 20:03:45
gender,Female,Female,Female,Female,Female
birth_year,2000,2000,1983,1983,1983


In [ ]:
# Export data 
df_final_3.to_pickle("step5_final_booking_prep.pkl")

In [ ]:
if TESTING == 1:
    if MODEL_TYPE_MEMBER ==1:
        df_final_3.to_pickle("booking_test_data_final_forMember.pkl")
    elif MODEL_TYPE_MEMBER ==0:
        df_final_3.to_pickle("booking_test_data_final_forBooking.pkl")
    else:
        print("Pls check the modeling purpose for the output testing.")
else: 
    df_final_3.to_pickle("booking_train_data_final.pkl")

<span class="burk">COMPLETE STEP 5</span>

## Step6 - Data Aggregate by Member

### Import Step 5 Finalised Booking Data

In [ ]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP5_BOOK = 0

In [ ]:
# Load data
if LOAD_INTERMEDIATE_STEP5_BOOK == 1:
    df_booking = read_pickle("step5_final_booking_prep.pkl")
else:
    df_booking = df_final_3

### Data Aggregation: New Features

#### Get tot numer of booking per member

In [ ]:
mem_agg1 = df_booking.groupby(["mem_id"])["booking_id"].count()
mem_agg1 = mem_agg1.reset_index()

In [ ]:
mem_agg1.rename(columns = {'mem_id' : 'mem_id', 'booking_id' : 'Mem_CountBooking'}, inplace = True)

In [ ]:
count_unique(mem_agg1)

,name,uniquecount
0,mem_id,5000
1,Mem_CountBooking,16


In [ ]:
mem_agg1["Mem_CountBooking"].value_counts()

1     2800
2     1000
3      506
4      257
5      174
6      109
7       66
8       26
9       21
11      10
10       9
13       9
12       9
19       2
15       1
14       1
Name: Mem_CountBooking, dtype: int64

#### Get mean for aggregation

In [ ]:
df_booking.info("deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10221 entries, 0 to 10220
Data columns (total 154 columns):
 #    Column                                     Dtype         
---   ------                                     -----         
 0    booking_id                                 object        
 1    mem_id                                     object        
 2    doctor_id                                  category      
 3    consult_waived_reason                      category      
 4    is_mc_issued                               int64         
 5    mc_days                                    float64       
 6    is_referral_letter_issued                  int64         
 7    booking_timestamp                          datetime64[ns]
 8    gender                                     category      
 9    birth_year                                 int64         
 10   diagnosis_code_list                        object        
 11   diagnosis_list                             object   

In [ ]:
select_list = [
     'mc_days',
     'count_diagnosis',
     'count_chronic',
     'count_medication',
     'count_repeatmed',
     'count_SymID',
     'age_yr']

In [ ]:
mem_agg2 = df_booking.groupby(["mem_id"])[select_list].mean()
mem_agg2 = mem_agg2.reset_index()

In [ ]:
mem_agg2 = mem_agg2.rename(columns={col: 'AVE_' + col
                        for col in mem_agg2.columns if col not in ['mem_id']})

In [ ]:
new_col2 = mem_agg2.columns.tolist()

In [ ]:
# Round off value to int
for col in new_col2[1:]:
    mem_agg2[col]  =  mem_agg2[col].apply(lambda x: int(x))

In [ ]:
mem_agg2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
AVE_mc_days,5000.0,0.9592,1.309304,0.0,0.0,1.0,1.0,7.0
AVE_count_diagnosis,5000.0,1.1324,0.379076,1.0,1.0,1.0,1.0,5.0
AVE_count_chronic,5000.0,0.0882,0.315025,0.0,0.0,0.0,0.0,3.0
AVE_count_medication,5000.0,1.8056,1.379489,0.0,1.0,2.0,3.0,8.0
AVE_count_repeatmed,5000.0,0.6926,0.467920,0.0,0.0,1.0,1.0,5.0
AVE_count_SymID,5000.0,1.7404,1.302823,0.0,1.0,1.0,2.0,10.0
AVE_age_yr,5000.0,36.0102,10.102777,17.0,29.0,34.0,41.0,102.0


#### Get max for aggregation

In [ ]:
mem_agg3 = df_booking.groupby(["mem_id"])[select_list].max()
mem_agg3 = mem_agg3.reset_index()

In [ ]:
mem_agg3 = mem_agg3.rename(columns={col: 'MAX_' + col
                        for col in mem_agg3.columns if col not in ['mem_id']})

In [ ]:
new_col3 = mem_agg3.columns.tolist()

In [ ]:
# Round off value to int
for col in new_col3[1:]:
    mem_agg3[col]  =  mem_agg3[col].apply(lambda x: int(x))

In [ ]:
mem_agg3.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MAX_mc_days,5000.0,1.3990,1.717437,0.0,0.0,1.0,2.0,14.0
MAX_count_diagnosis,5000.0,1.3562,0.620160,1.0,1.0,1.0,2.0,6.0
MAX_count_chronic,5000.0,0.2034,0.492419,0.0,0.0,0.0,0.0,4.0
MAX_count_medication,5000.0,2.4190,1.674754,0.0,1.0,2.0,3.0,14.0
MAX_count_repeatmed,5000.0,0.8558,0.373409,0.0,1.0,1.0,1.0,5.0
MAX_count_SymID,5000.0,2.2334,1.528853,0.0,1.0,2.0,3.0,10.0
MAX_age_yr,5000.0,36.0102,10.102777,17.0,29.0,34.0,41.0,102.0


#### Get min for aggregation

In [ ]:
mem_agg4 = df_booking.groupby(["mem_id"])[select_list].min()
mem_agg4 = mem_agg4.reset_index()

In [ ]:
mem_agg4 = mem_agg4.rename(columns={col: 'MIN_' + col
                        for col in mem_agg4.columns if col not in ['mem_id']})

In [ ]:
new_col4 = mem_agg4.columns.tolist()

In [ ]:
# Round off value to int
for col in new_col4[1:]:
    mem_agg4[col]  =  mem_agg4[col].apply(lambda x: int(x))

In [ ]:
mem_agg4.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
MIN_mc_days,5000.0,0.8076,1.271891,0.0,0.0,0.0,1.0,7.0
MIN_count_diagnosis,5000.0,1.1132,0.354416,1.0,1.0,1.0,1.0,5.0
MIN_count_chronic,5000.0,0.0792,0.299575,0.0,0.0,0.0,0.0,3.0
MIN_count_medication,5000.0,1.5154,1.435749,0.0,0.0,1.0,2.0,8.0
MIN_count_repeatmed,5000.0,0.6912,0.468494,0.0,0.0,1.0,1.0,5.0
MIN_count_SymID,5000.0,1.5684,1.307235,0.0,1.0,1.0,2.0,10.0
MIN_age_yr,5000.0,36.0102,10.102777,17.0,29.0,34.0,41.0,102.0


In [ ]:
mem_agg4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   mem_id                5000 non-null   object
 1   MIN_mc_days           5000 non-null   int64 
 2   MIN_count_diagnosis   5000 non-null   int64 
 3   MIN_count_chronic     5000 non-null   int64 
 4   MIN_count_medication  5000 non-null   int64 
 5   MIN_count_repeatmed   5000 non-null   int64 
 6   MIN_count_SymID       5000 non-null   int64 
 7   MIN_age_yr            5000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 312.6+ KB


#### Check Age

In [ ]:
dff_max_mean_age = mem_agg3["MAX_age_yr"] - mem_agg2["AVE_age_yr"]

In [ ]:
sum(dff_max_mean_age>0)

0

In [ ]:
dff_max_min_age = mem_agg3["MAX_age_yr"] - mem_agg4["MIN_age_yr"]

In [ ]:
sum(dff_max_min_age>0)

0

In [ ]:
# Conclude: Age is consistet, can drop Min and Max
mem_agg3.drop(["MAX_age_yr"], axis=1, inplace=True)
mem_agg4.drop(["MIN_age_yr"], axis=1, inplace=True)


In [ ]:
mem_agg2.rename(columns = {'AVE_age_yr' : 'age_yr'}, inplace = True)

In [ ]:
mem_agg2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   mem_id                5000 non-null   object
 1   AVE_mc_days           5000 non-null   int64 
 2   AVE_count_diagnosis   5000 non-null   int64 
 3   AVE_count_chronic     5000 non-null   int64 
 4   AVE_count_medication  5000 non-null   int64 
 5   AVE_count_repeatmed   5000 non-null   int64 
 6   AVE_count_SymID       5000 non-null   int64 
 7   age_yr                5000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 312.6+ KB


#### Get count for aggregation for Waived_Reason

In [ ]:
mem_agg5 = df_booking.groupby(["mem_id",'consult_waived_reason'])["booking_id"].count()
mem_agg5 = mem_agg5.reset_index()

In [ ]:
mem_agg5.rename(columns = {'mem_id' : 'mem_id', "consult_waived_reason": "consult_waived_reason", 
                           'booking_id' : 'Reason_CountBooking'}, inplace = True)

In [ ]:
mem_agg5.head()

,mem_id,consult_waived_reason,Reason_CountBooking
0,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,NA,1
1,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,FOLLOW UP,0
2,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,HEALTH SCREENING REVIEW,0
3,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,OTHERS,0
4,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,REFERRAL,0


In [ ]:
mem_agg5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   mem_id                 35000 non-null  object  
 1   consult_waived_reason  35000 non-null  category
 2   Reason_CountBooking    35000 non-null  int64   
dtypes: category(1), int64(1), object(1)
memory usage: 581.5+ KB


In [ ]:
mem_agg5_2 = pd.pivot_table(mem_agg5, index="mem_id", columns='consult_waived_reason', 
                            values='Reason_CountBooking')

In [ ]:
mem_agg5_2 = mem_agg5_2.reset_index()

In [ ]:
mem_agg5_2.head(10)

consult_waived_reason,mem_id,NA,FOLLOW UP,HEALTH SCREENING REVIEW,OTHERS,REFERRAL,UNSUITABLE FOR TELECONSULT,WHITECOAT FAMILY
0,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,1,0,0,0,0,0,0
1,001205F3-C949-4A64-B7D8-8BB38201627C,2,0,0,0,0,0,0
2,0012A643-20F7-46B7-8A9E-9063E7A5CAA9,1,0,0,0,0,0,0
3,0026B107-D521-4150-94E1-C488C9F6D97C,7,0,0,0,0,0,0
4,003AAC06-DA74-4C1C-AC73-86BBA0357155,1,0,0,0,0,0,0
5,005229E8-0858-42E3-9C70-E07D21B6231A,1,0,0,0,0,0,0
6,005C4ED2-EFCC-4305-8623-09A640AA1D9C,4,1,0,0,0,0,0
7,005C9BB7-7428-4B54-9858-54FA85271A15,1,0,0,0,0,0,0
8,00715C95-A3BF-464E-B671-8BF601F1E0CB,2,0,0,1,0,0,0
9,007972F8-5E5A-41A7-ADA7-F28ED510639A,5,0,0,0,0,0,0


In [ ]:
mem_agg5_2 = mem_agg5_2.rename(columns={col: 'WaviedCount_' + col
                        for col in mem_agg5_2.columns if col not in ['mem_id']})

In [ ]:
mem_agg5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   mem_id                                  5000 non-null   object
 1   WaviedCount_NA                          5000 non-null   int64 
 2   WaviedCount_FOLLOW UP                   5000 non-null   int64 
 3   WaviedCount_HEALTH SCREENING REVIEW     5000 non-null   int64 
 4   WaviedCount_OTHERS                      5000 non-null   int64 
 5   WaviedCount_REFERRAL                    5000 non-null   int64 
 6   WaviedCount_UNSUITABLE FOR TELECONSULT  5000 non-null   int64 
 7   WaviedCount_WHITECOAT FAMILY            5000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 312.6+ KB


In [ ]:
mem_agg5_2.columns = mem_agg5_2.columns.str.replace(' ','_')

In [ ]:
mem_agg5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   mem_id                                  5000 non-null   object
 1   WaviedCount_NA                          5000 non-null   int64 
 2   WaviedCount_FOLLOW_UP                   5000 non-null   int64 
 3   WaviedCount_HEALTH_SCREENING_REVIEW     5000 non-null   int64 
 4   WaviedCount_OTHERS                      5000 non-null   int64 
 5   WaviedCount_REFERRAL                    5000 non-null   int64 
 6   WaviedCount_UNSUITABLE_FOR_TELECONSULT  5000 non-null   int64 
 7   WaviedCount_WHITECOAT_FAMILY            5000 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 312.6+ KB


In [ ]:
mem_agg5_2.columns

Index(['mem_id', 'WaviedCount_NA', 'WaviedCount_FOLLOW_UP',
       'WaviedCount_HEALTH_SCREENING_REVIEW', 'WaviedCount_OTHERS',
       'WaviedCount_REFERRAL', 'WaviedCount_UNSUITABLE_FOR_TELECONSULT',
       'WaviedCount_WHITECOAT_FAMILY'],
      dtype='object', name='consult_waived_reason')

#### Get count for aggregation for ConsultHr

In [ ]:
mem_agg6 = df_booking.groupby(["mem_id",'consult_hr'])["booking_id"].count()
mem_agg6 = mem_agg6.reset_index()

In [ ]:
mem_agg6.rename(columns = {'mem_id' : 'mem_id', "consult_hr": "consult_hr", 
                           'booking_id' : 'ConsultHr_CountBooking'}, inplace = True)

In [ ]:
mem_agg6_2 = pd.pivot_table(mem_agg6, index="mem_id", columns='consult_hr', 
                            values='ConsultHr_CountBooking')

In [ ]:
mem_agg6_2 = mem_agg6_2.reset_index()

In [ ]:
mem_agg6_2.head()

consult_hr,mem_id,0,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,001205F3-C949-4A64-B7D8-8BB38201627C,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0012A643-20F7-46B7-8A9E-9063E7A5CAA9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0026B107-D521-4150-94E1-C488C9F6D97C,0,0,0,0,1,0,1,4,1,0,0,0,0,0,0,0,0
4,003AAC06-DA74-4C1C-AC73-86BBA0357155,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
mem_agg6_2 = mem_agg6_2.rename(columns={col: 'ConsultHr_' + str(col)
                        for col in mem_agg6_2.columns if col not in ['mem_id']})

In [ ]:
mem_agg6_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mem_id        5000 non-null   object
 1   ConsultHr_0   5000 non-null   int64 
 2   ConsultHr_8   5000 non-null   int64 
 3   ConsultHr_9   5000 non-null   int64 
 4   ConsultHr_10  5000 non-null   int64 
 5   ConsultHr_11  5000 non-null   int64 
 6   ConsultHr_12  5000 non-null   int64 
 7   ConsultHr_13  5000 non-null   int64 
 8   ConsultHr_14  5000 non-null   int64 
 9   ConsultHr_15  5000 non-null   int64 
 10  ConsultHr_16  5000 non-null   int64 
 11  ConsultHr_17  5000 non-null   int64 
 12  ConsultHr_18  5000 non-null   int64 
 13  ConsultHr_19  5000 non-null   int64 
 14  ConsultHr_20  5000 non-null   int64 
 15  ConsultHr_21  5000 non-null   int64 
 16  ConsultHr_22  5000 non-null   int64 
 17  ConsultHr_23  5000 non-null   int64 
dtypes: int64(17), object(1)
memory usage: 703.2+ KB


In [ ]:
mem_agg6_2.columns

Index(['mem_id', 'ConsultHr_0', 'ConsultHr_8', 'ConsultHr_9', 'ConsultHr_10',
       'ConsultHr_11', 'ConsultHr_12', 'ConsultHr_13', 'ConsultHr_14',
       'ConsultHr_15', 'ConsultHr_16', 'ConsultHr_17', 'ConsultHr_18',
       'ConsultHr_19', 'ConsultHr_20', 'ConsultHr_21', 'ConsultHr_22',
       'ConsultHr_23'],
      dtype='object', name='consult_hr')

#### Get sum for aggregation for Dummy Variable

In [ ]:
filter_col_diag = [col for col in df_booking if col.startswith('diag_')]

In [ ]:
filter_col_chronic = [col for col in df_booking if col.startswith('chronic_')]

In [ ]:
filter_col_symp = [col for col in df_booking if col.startswith('symp_')]

In [ ]:
fitler_col_other_bool =  ["is_mc_issued", "is_referral_letter_issued", "Target_chronic"]

In [ ]:
select_list_dummy = fitler_col_other_bool + filter_col_diag + filter_col_chronic + filter_col_symp

In [ ]:
# must change to numeric to do sum
for col in select_list_dummy:
    df_booking[col] = df_booking[col].astype("int")

In [ ]:
mem_agg7 = df_booking.groupby(["mem_id"])[select_list_dummy].sum()
mem_agg7 = mem_agg7.reset_index()

In [ ]:
mem_agg7 = mem_agg7.rename(columns={col: 'CountVisit_' + col
                        for col in mem_agg7.columns if col not in ['mem_id']})

In [ ]:
mem_agg7.columns = mem_agg7.columns.str.replace(' ','_')

Reminder: The count of ChronicVisit, e.g. CountVisit_Target_chronic, refers to visits with Chronic status marked, it idoes not related to Chronic diagnostic the visit.

#### Get first dates related aggregation

In [ ]:
book_firstdate = df_booking.groupby(["mem_id"])["booking_timestamp"].min()
book_firstdate = book_firstdate.reset_index()

In [ ]:
book_firstdate.rename(columns = {'mem_id' : 'mem_id', 'booking_timestamp' : 'FirstBookDate'}, inplace = True)

In [ ]:
book_firstdate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   mem_id         5000 non-null   object        
 1   FirstBookDate  5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 78.2+ KB


In [ ]:
book_firstdate.head()

,mem_id,FirstBookDate
0,0004A803-9EAB-4E3E-ACA6-F890F3805CFB,2021-10-18 11:26:56
1,001205F3-C949-4A64-B7D8-8BB38201627C,2021-10-14 20:35:19
2,0012A643-20F7-46B7-8A9E-9063E7A5CAA9,2021-11-24 08:22:11
3,0026B107-D521-4150-94E1-C488C9F6D97C,2021-01-11 15:13:40
4,003AAC06-DA74-4C1C-AC73-86BBA0357155,2021-07-07 14:30:18


#### Get last dates related aggregation

In [ ]:
book_lastdate = df_booking.groupby(["mem_id"])["booking_timestamp"].max()
book_lastdate = book_lastdate.reset_index()

In [ ]:
book_lastdate.rename(columns = {'mem_id' : 'mem_id', 'booking_timestamp' : 'LastBookDate'}, inplace = True)

#### Get Mode for aggregation Consult Waived Reason

In [ ]:
mem_agg8 = df_booking.groupby(["mem_id"])["consult_waived_reason"].apply(lambda x: x.value_counts().index[0])
mem_agg8 = mem_agg8.reset_index() 

In [ ]:
mem_agg8.rename(columns = {'mem_id' : 'mem_id', 'consult_waived_reason' : 'Mode_consult_waived_reason'}, inplace = True)

In [ ]:
mem_agg8["Mode_consult_waived_reason"].value_counts()

NA                            4791
HEALTH SCREENING REVIEW        124
OTHERS                          44
UNSUITABLE FOR TELECONSULT      22
FOLLOW UP                       14
WHITECOAT FAMILY                 5
Name: Mode_consult_waived_reason, dtype: int64

#### Get Mode for aggregation Consultation Hour

In [ ]:
mem_agg9 = df_booking.groupby(["mem_id"])["consult_hr"].apply(lambda x: x.value_counts().index[0])
mem_agg9 = mem_agg9.reset_index()

In [ ]:
mem_agg9.rename(columns = {'mem_id' : 'mem_id', 'consult_hr' : 'Mode_consult_hr'}, inplace = True)

#### Get gender for member

In [ ]:
mem_gender = df_booking.groupby(["mem_id"])["gender"].apply(lambda x: x.value_counts().index[0])
mem_gender = mem_gender.reset_index() 

In [ ]:
mem_gender["gender"].value_counts()

Female     3016
Male       1774
Unknown     210
Name: gender, dtype: int64

### Merge Aggregation

* mem_agg1 : Mem_CountBooking
* mem_agg2 : 'mem_id', 'AVE_mc_days', 'AVE_count_diagnosis', 'AVE_count_chronic',
       'AVE_count_medication', 'AVE_count_repeatmed', 'AVE_count_SymID',
       'age_yr'
* mem_agg3 : 'mem_id', 'MIN_mc_days', 'MIN_count_diagnosis', 'MIN_count_chronic',
       'MIN_count_medication', 'MIN_count_repeatmed', 'MIN_count_SymID'
* mem_agg4 : 'mem_id', 'MIN_mc_days', 'MIN_count_diagnosis', 'MIN_count_chronic',
       'MIN_count_medication', 'MIN_count_repeatmed', 'MIN_count_SymID'
* mem_agg5_2 : 'WaviedCount_NA','WaviedCount_FOLLOW_UP','WaviedCount_HEALTH_SCREENING_REVIEW',
        'WaviedCount_OTHERS','WaviedCount_REFERRAL','WaviedCount_UNSUITABLE_FOR_TELECONSULT','WaviedCount_WHITECOAT_FAMILY'
* mem_agg6_2: 'ConsultHr_0','ConsultHr_8','ConsultHr_9','ConsultHr_10','ConsultHr_11',
         'ConsultHr_12','ConsultHr_13','ConsultHr_14','ConsultHr_15','ConsultHr_16',
         'ConsultHr_17','ConsultHr_18','ConsultHr_19','ConsultHr_20','ConsultHr_21',
         'ConsultHr_22','ConsultHr_23'     
* mem_agg7: 'CountVisit_is_mc_issued',
         'CountVisit_is_referral_letter_issued',
         'CountVisit_diag_H57',
         'CountVisit_diag_M23',
         'CountVisit_diag_T88',
         'CountVisit_diag_L20',
         'CountVisit_diag_R51',
         'CountVisit_diag_N94',
         'CountVisit_diag_G44',
         'CountVisit_diag_M79',
         'CountVisit_diag_J02',
         'CountVisit_diag_J06',
         'CountVisit_diag_G43',
         'CountVisit_diag_R21',
         'CountVisit_diag_M62',
         'CountVisit_diag_A09',
         'CountVisit_diag_J30',
         'CountVisit_diag_L30',
         'CountVisit_diag_T78',
         'CountVisit_diag_U07',
         'CountVisit_diag_K29',
         'CountVisit_diag_B35',
         'CountVisit_diag_K30',
         'CountVisit_diag_H81',
         'CountVisit_diag_N39',
         'CountVisit_diag_G47',
         'CountVisit_diag_Z76',
         'CountVisit_diag_K59',
         'CountVisit_diag_H10',
         'CountVisit_diag_R42',
         'CountVisit_diag_K21',
         'CountVisit_diag_S90',
         'CountVisit_diag_R50',
         'CountVisit_diag_U12',
         'CountVisit_diag_N92',
         'CountVisit_diag_M54',
         'CountVisit_diag_S60',
         'CountVisit_diag_I10',
         'CountVisit_diag_L03',
         'CountVisit_diag_K64',
         'CountVisit_diag_L98',
         'CountVisit_diag_B37',
         'CountVisit_diag_L73',
         'CountVisit_diag_H00',
         'CountVisit_diag_N76',
         'CountVisit_diag_J01',
         'CountVisit_diag_J45',
         'CountVisit_diag_E78',
         'CountVisit_diag_L02',
         'CountVisit_diag_R52',
         'CountVisit_diag_K12',
         'CountVisit_diag_K58',
         'CountVisit_diag_K07',
         'CountVisit_diag_L70',
         'CountVisit_diag_Z71',
         'CountVisit_diag_S63',
         'CountVisit_diag_M25',
         'CountVisit_diag_K05',
         'CountVisit_diag_E03',
         'CountVisit_diag_L50',
         'CountVisit_diag_W57',
         'CountVisit_diag_H93',
         'CountVisit_diag_H01',
         'CountVisit_diag_E11',
         'CountVisit_diag_S13',
         'CountVisit_diag_F41',
         'CountVisit_diag_R53',
         'CountVisit_diag_O91',
         'CountVisit_diag_H02',
         'CountVisit_diag_R11',
         'CountVisit_diag_S93',
         'CountVisit_diag_L24',
         'CountVisit_diag_J39',
         'CountVisit_diag_J32',
         'CountVisit_diag_B00',
         'CountVisit_diag_L64',
         'CountVisit_diag_Y42',
         'CountVisit_diag_R10',
         'CountVisit_diag_M10',
         'CountVisit_diag_M75',
         'CountVisit_diag_Z02',
         'CountVisit_diag_J03',
         'CountVisit_diag_R07',
         'CountVisit_diag_K13',
         'CountVisit_diag_Z30',
         'CountVisit_diag_P39',
         'CountVisit_diag_R22',
         'CountVisit_diag_H60',
         'CountVisit_diag_B02',
         'CountVisit_diag_L29',
         'CountVisit_diag_E05',
         'CountVisit_diag_N64',
         'CountVisit_diag_R05',
         'CountVisit_diag_L60',
         'CountVisit_diag_S80',
         'CountVisit_diag_M65',
         'CountVisit_diag_J22',
         'CountVisit_diag_O21',
         'CountVisit_diag_M72',
         'CountVisit_diag_L23',
         'CountVisit_diag_K52',
         'CountVisit_chronic_NoneChronic',
         'CountVisit_chronic_Atopic dermatitis',
         'CountVisit_chronic_Migraine',
         'CountVisit_chronic_Vasomotor and allergic rhinitis',
         'CountVisit_chronic_Gastro-oesophageal reflux disease',
         'CountVisit_chronic_Hypertension',
         'CountVisit_chronic_Asthma',
         'CountVisit_chronic_Hyperlipidaemia',
         'CountVisit_chronic_Thyroid',
         'CountVisit_chronic_Diabetes',
         'CountVisit_chronic_Gout',
         'CountVisit_symp_others',
         'CountVisit_symp_medical referral',
         'CountVisit_symp_headache',
         'CountVisit_symp_giddiness',
         'CountVisit_symp_fever',
         'CountVisit_symp_N',
         'CountVisit_symp_cold sores',
         'CountVisit_symp_rash',
         'CountVisit_symp_sore throat',
         'CountVisit_symp_flu',
         'CountVisit_symp_cough',
         'CountVisit_symp_cold',
         'CountVisit_symp_vomiting',
         'CountVisit_symp_diarrhoea',
         'CountVisit_symp_chronic medicine top-up',
         'CountVisit_symp_chronic illness management',
         'CountVisit_symp_discuss lab results',
         'CountVisit_Target_chronic'
* book_firstdate: 'FirstBookDate'
* book_lastdate: 'LastBookDate'
* mem_agg8: 'Mode_consult_waived_reason'
* mem_agg9: "Mode_consult_hr"
* mem_gender: gender
 

In [ ]:
# initiate dataframe
new_df = pd.DataFrame() 

In [ ]:
new_df = pd.merge(mem_gender,mem_agg1, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg2, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg3, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg4, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg5_2, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg6_2, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg7, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg8, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, mem_agg9, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, book_firstdate, how="inner", on=["mem_id"])

In [ ]:
new_df = pd.merge(new_df, book_lastdate, how="inner", on=["mem_id"])

In [ ]:
new_df.shape

(5000, 181)

### Add Feature - Diff between Last and First Date

In [ ]:
new_df["DiffDays_Vist"] = new_df["LastBookDate"]-new_df["FirstBookDate"]

In [ ]:
new_df["DiffDays_Vist"]  = new_df["DiffDays_Vist"].apply(lambda x: abs(x.days))

### Add Feature - Average Visit Freq

In [ ]:
new_df["Ave_VisitGapDays"] = new_df["DiffDays_Vist"] / new_df["Mem_CountBooking"]

In [ ]:
new_df["Ave_VisitGapDays"]  = new_df["Ave_VisitGapDays"].apply(lambda x: int(x))

### Create Targets

In [ ]:
new_df["Target_Chronic"] = new_df["CountVisit_Target_chronic"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Dermatitis"] = new_df["CountVisit_chronic_Atopic_dermatitis"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Migraine"] = new_df["CountVisit_chronic_Migraine"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Allergic"] = new_df["CountVisit_chronic_Vasomotor_and_allergic_rhinitis"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Gastro"] = new_df["CountVisit_chronic_Gastro-oesophageal_reflux_disease"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Hypertension"] = new_df["CountVisit_chronic_Hypertension"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Asthma"] = new_df["CountVisit_chronic_Asthma"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Hyperlipidaemia"] = new_df["CountVisit_chronic_Hyperlipidaemia"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Thyroid"] = new_df["CountVisit_chronic_Thyroid"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Diabetes"] = new_df["CountVisit_chronic_Diabetes"].apply(lambda x : 1 if x >0 else 0)

In [ ]:
new_df["Target_Gout"] = new_df["CountVisit_chronic_Gout"].apply(lambda x : 1 if x >0 else 0)

### Export Prep Data at Member level

In [ ]:
# Export data 
new_df.to_pickle("step6_member_prep.pkl")

## Step7 -  Define Group Targets

### Import Step 6 Aggregated Member Data

In [ ]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP6_MEM = 0

In [ ]:
# Load data
if LOAD_INTERMEDIATE_STEP6_MEM == 1:
    new_df = read_pickle("step6_member_prep.pkl")
else:
    new_df = new_df

### Define 3 High - Diabetes, Hyperlipidaemia, Hypertension

In [ ]:
new_df["temp_3High"] = new_df["Target_Hyperlipidaemia"] + new_df["Target_Diabetes"] + new_df["Target_Hypertension"]
new_df["Target_3High"] = new_df["temp_3High"].apply(lambda x: 1 if x>0 else 0)

In [ ]:
new_df.Target_Diabetes.value_counts()

0    4971
1      29
Name: Target_Diabetes, dtype: int64

In [ ]:
new_df.Target_Hyperlipidaemia.value_counts()

0    4910
1      90
Name: Target_Hyperlipidaemia, dtype: int64

In [ ]:
new_df.Target_Hypertension.value_counts()

0    4902
1      98
Name: Target_Hypertension, dtype: int64

In [ ]:
new_df.Target_3High.value_counts()

0    4845
1     155
Name: Target_3High, dtype: int64

In [ ]:
new_df = new_df.drop(columns=["temp_3High"])

### Define Allergic_Dermatitis_Asthma

In [ ]:
# Target for allergic related - Atopic_dermatitis, Vasomotor_and_allergic_rhinitic, Asthma
new_df["temp_Allergic_Dermatitis_Asthma"] = new_df["Target_Dermatitis"] + new_df["Target_Allergic"] + new_df["Target_Asthma"]
new_df["Target_Allergic_Dermatitis_Asthma"] = new_df["temp_Allergic_Dermatitis_Asthma"].apply(lambda x: 1 if x>0 else 0)

In [ ]:
new_df.Target_Dermatitis.value_counts()

0    4842
1     158
Name: Target_Dermatitis, dtype: int64

In [ ]:
new_df.Target_Allergic.value_counts()

0    4747
1     253
Name: Target_Allergic, dtype: int64

In [ ]:
new_df.Target_Asthma.value_counts()

0    4913
1      87
Name: Target_Asthma, dtype: int64

In [ ]:
new_df["temp_Allergic_Dermatitis_Asthma"].value_counts()

0    4566
1     375
2      54
3       5
Name: temp_Allergic_Dermatitis_Asthma, dtype: int64

In [ ]:
new_df["Target_Allergic_Dermatitis_Asthma"].value_counts()

0    4566
1     434
Name: Target_Allergic_Dermatitis_Asthma, dtype: int64

In [ ]:
new_df = new_df.drop(columns=["temp_Allergic_Dermatitis_Asthma"])

### Define Migraine_Gastro_Thyroid_Gou

In [ ]:
new_df["temp_Migraine_Gastro_Thyroid_Gout"] = new_df["Target_Migraine"] + new_df["Target_Gastro"] + new_df["Target_Thyroid"] + new_df["Target_Gout"]
new_df["Target_Migraine_Gastro_Thyroid_Gout"] = new_df["temp_Migraine_Gastro_Thyroid_Gout"].apply(lambda x: 1 if x>0 else 0)

In [ ]:
new_df.Target_Migraine.value_counts()

0    4867
1     133
Name: Target_Migraine, dtype: int64

In [ ]:
new_df.Target_Gastro.value_counts()

0    4844
1     156
Name: Target_Gastro, dtype: int64

In [ ]:
new_df.Target_Thyroid.value_counts()

0    4973
1      27
Name: Target_Thyroid, dtype: int64

In [ ]:
new_df.Target_Gout.value_counts()

0    4920
1      80
Name: Target_Gout, dtype: int64

In [ ]:
new_df["Target_Migraine_Gastro_Thyroid_Gout"].value_counts()

0    4624
1     376
Name: Target_Migraine_Gastro_Thyroid_Gout, dtype: int64

In [ ]:
new_df = new_df.drop(columns=["temp_Migraine_Gastro_Thyroid_Gout"])

### Group Consultancy Hour

Group Consultancy Hour based on PCA and MCA analysis

In [ ]:
new_df["ConsultHr_8to10"] = new_df["ConsultHr_8"] + new_df["ConsultHr_9"] + new_df["ConsultHr_10"]

In [ ]:
new_df["ConsultHr_11to13"] = new_df["ConsultHr_11"] + new_df["ConsultHr_12"] + new_df["ConsultHr_13"]

In [ ]:
new_df["ConsultHr_14to17"] = new_df["ConsultHr_14"] + new_df["ConsultHr_15"] + new_df["ConsultHr_16"] + new_df["ConsultHr_17"]

In [ ]:
new_df["ConsultHr_18to21"] = new_df["ConsultHr_18"] + new_df["ConsultHr_19"] + new_df["ConsultHr_20"] + new_df["ConsultHr_21"]

In [ ]:
new_df["ConsultHr_22to23"] = new_df["ConsultHr_22"] + new_df["ConsultHr_23"] 

### Export Prep Member Data with Chronic Grouping

In [ ]:
# Export data 
new_df.to_pickle("step7_member_grp_prep.pkl")

## Step8 - Finalise Member Data

As some of the syptoms are very highly correalated. PCA is performed on the highly corrected variables using JMP software to generate out the rules for deriving the principal components. 

### Import Step 7 Finalised Member Data with Chronic Grouping

In [ ]:
# Change the parameter if intend to load intermediate data instead of runnning preceeding scripts
LOAD_INTERMEDIATE_STEP7_MEM_GP = 0

In [ ]:
# Load data
if LOAD_INTERMEDIATE_STEP7_MEM_GP == 1:
    df_mem = read_pickle("step7_member_grp_prep.pkl")
else:
    df_mem = new_df

### Create New Features for Principal Components for Symptoms

new columns = ['Prin1_sym_common',
       'Prin2_sym_rash', 'Prin3_sym_headache', 'Prin4_sym_diarrhea']

#### Principal Component1 - Common 

Formula from JMP: 
Prin1_sym_common = 0.126922208999099 * :CountVisit_symp_headache + 0.134938007927841 *
:CountVisit_symp_giddiness + 0.457991577692383 * :CountVisit_symp_cold_sores
+0.448844403126012 * :CountVisit_symp_rash + 0.954160558888485 *
:CountVisit_symp_sore_throat + 0.960039882968821 * :CountVisit_symp_flu
+0.966269306714024 * :CountVisit_symp_cough + 0.744793202425646 *
:CountVisit_symp_cold + 0.111674590804738 * :CountVisit_symp_vomiting
+0.105420843334157 * :CountVisit_symp_diarrhoea + (-1.08200575730171)

In [ ]:
df_mem["Prin1_sym_common"] = 0.126922208999099 * df_mem["CountVisit_symp_headache"] + \
                             0.134938007927841 * df_mem["CountVisit_symp_giddiness"] + \
                             0.457991577692383 * df_mem["CountVisit_symp_cold_sores"] + \
                             0.448844403126012 * df_mem["CountVisit_symp_rash"] + \
                             0.954160558888485 * df_mem["CountVisit_symp_sore_throat"] + \
                             0.960039882968821 * df_mem["CountVisit_symp_flu"] + \
                             0.966269306714024 * df_mem["CountVisit_symp_cough"] + \
                             0.744793202425646 * df_mem["CountVisit_symp_cold"] + \
                             0.111674590804738 * df_mem["CountVisit_symp_vomiting"] + \
                             0.105420843334157 * df_mem["CountVisit_symp_diarrhoea"] - 1.08200575730171

#### Principal Component - Rash

Formula from JMP: 
Prin2_sym_rash = 0.15012952272156 * :CountVisit_symp_headache + 0.157099331295396 *
:CountVisit_symp_giddiness + 1.4198008192801 * :CountVisit_symp_cold_sores
+1.39667010805915 * :CountVisit_symp_rash + -0.509538387575815 *
:CountVisit_symp_sore_throat + -0.51973978746157 * :CountVisit_symp_flu +
-0.518711617511583 * :CountVisit_symp_cough + 0.286064554630894 *
:CountVisit_symp_cold + 0.281708989416286 * :CountVisit_symp_vomiting
+0.268761673670889 * :CountVisit_symp_diarrhoea + (-0.399412355479641)

In [ ]:
df_mem["Prin2_sym_rash"] = 0.15012952272156 * df_mem["CountVisit_symp_headache"] + \
                            0.157099331295396 * df_mem["CountVisit_symp_giddiness"] + \
                            1.4198008192801 * df_mem["CountVisit_symp_cold_sores"] + \
                            1.39667010805915 * df_mem["CountVisit_symp_rash"] - \
                            0.509538387575815 * df_mem["CountVisit_symp_sore_throat"] - \
                            0.51973978746157 * df_mem["CountVisit_symp_flu"] - \
                            0.518711617511583 * df_mem["CountVisit_symp_cough"] + \
                            0.286064554630894 * df_mem["CountVisit_symp_cold"] + \
                            0.281708989416286 * df_mem["CountVisit_symp_vomiting"] + \
                            0.268761673670889 * df_mem["CountVisit_symp_diarrhoea"] -0.399412355479641

#### Principal Component - Headache 

Formula from JMP: 
Prin3_sym_headache = 0.53393749284155 * :CountVisit_symp_headache + 0.549256657956352 *
:CountVisit_symp_giddiness + -0.540351073738283 * :CountVisit_symp_cold_sores +
-0.532878377771104 * :CountVisit_symp_rash + -0.0177007121608483 *
:CountVisit_symp_sore_throat + -0.0186518204119694 * :CountVisit_symp_flu +
-0.0183140476024677 * :CountVisit_symp_cough + -0.233303688008869 *
:CountVisit_symp_cold + 0.786678691868385 * :CountVisit_symp_vomiting
+0.762367704391997 * :CountVisit_symp_diarrhoea + (-0.711589799772594)

In [ ]:
df_mem["Prin3_sym_headache"] = 0.53393749284155 * df_mem["CountVisit_symp_headache"] + \
                                0.549256657956352 * df_mem["CountVisit_symp_giddiness"] - \
                                0.540351073738283 * df_mem["CountVisit_symp_cold_sores"] - \
                                0.532878377771104 * df_mem["CountVisit_symp_rash"] - \
                                0.0177007121608483 * df_mem["CountVisit_symp_sore_throat"] - \
                                0.0186518204119694 * df_mem["CountVisit_symp_flu"] - \
                                0.0183140476024677 * df_mem["CountVisit_symp_cough"] - \
                                0.233303688008869 * df_mem["CountVisit_symp_cold"] + \
                                0.786678691868385 * df_mem["CountVisit_symp_vomiting"] + \
                                0.762367704391997 * df_mem["CountVisit_symp_diarrhoea"] -0.711589799772594

#### Principal Component - Diarrhea

Formula from JMP: 
Prin4_sym_diarrhea = -0.571563615572338 * :CountVisit_symp_headache + -0.587138492269664 *
:CountVisit_symp_giddiness + -0.0255056350216958 * :CountVisit_symp_cold_sores +
-0.0200870916998447 * :CountVisit_symp_rash + 0.0708239066134195 *
:CountVisit_symp_sore_throat + 0.0595575562025494 * :CountVisit_symp_flu
+0.0650590997464691 * :CountVisit_symp_cough + 0.0197435600586072 *
:CountVisit_symp_cold + 0.835757956281788 * :CountVisit_symp_vomiting
+0.807263138560611 * :CountVisit_symp_diarrhoea + 0.0422774813710458

In [ ]:
df_mem["Prin4_sym_diarrhea"] = -0.571563615572338 * df_mem["CountVisit_symp_headache"] - \
                                0.587138492269664 * df_mem["CountVisit_symp_giddiness"] - \
                                0.0255056350216958 * df_mem["CountVisit_symp_cold_sores"] - \
                                0.0200870916998447 * df_mem["CountVisit_symp_rash"] + \
                                0.0708239066134195 * df_mem["CountVisit_symp_sore_throat"] + \
                                0.0595575562025494 * df_mem["CountVisit_symp_flu"] + \
                                0.0650590997464691 * df_mem["CountVisit_symp_cough"] + \
                                0.0197435600586072 * df_mem["CountVisit_symp_cold"] + \
                                0.835757956281788 * df_mem["CountVisit_symp_vomiting"] + \
                                0.807263138560611 * df_mem["CountVisit_symp_diarrhoea"] + 0.0422774813710458

### Export Final Member Data for modeling or Testing

In [ ]:
df_mem.info("deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 206 columns):
 #    Column                                                Dtype         
---   ------                                                -----         
 0    mem_id                                                object        
 1    gender                                                object        
 2    Mem_CountBooking                                      int64         
 3    AVE_mc_days                                           int64         
 4    AVE_count_diagnosis                                   int64         
 5    AVE_count_chronic                                     int64         
 6    AVE_count_medication                                  int64         
 7    AVE_count_repeatmed                                   int64         
 8    AVE_count_SymID                                       int64         
 9    age_yr                                                int64  

In [ ]:
df_mem.shape

(5000, 206)

In [ ]:
# Export data 
df_mem.to_pickle("step8_final_member.pkl")

In [ ]:
if TESTING == 1:
    df_mem.to_pickle("member_test_data_final.pkl")
else: 
    df_mem.to_pickle("member_train_data_final.pkl")

@end